# Classificação de Texto e Reranqueadores

Desenvolvimento de um buscador Simples: Booleano, TF-IDF, BM25

Tópicos abordados: Indexação, Bag-of-Words, TF-IDF, BM25

Aula 1 - [Unicamp - IA368DD: Deep Learning aplicado a sistemas de busca.](https://www.cpg.feec.unicamp.br/cpg/lista/caderno_horario_show.php?id=1779)

Autor: Marcus Vinícius Borela de Castro

[Repositório no github](https://github.com/marcusborela/deep_learning_em_buscas_unicamp)

[Link para chat de apoio com WebChatGPT](https://github.com/marcusborela/deep_learning_em_buscas_unicamp/blob/main/chat/CG%20uso%20aula_2_classificacao_de_texto_e_reranqueador.md)

[![Open In Colab latest github version](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/marcusborela/deep_learning_em_buscas_unicamp/blob/main/code/aula_2_classificacao_de_texto_e_reranqueador.ipynb) [Open In Colab latest github version]

# Enunciado exercício

Aula 2 - Notebook: Classificação de Texto e Reranqueadores

Data de entrega: 15 de mar., 23:59

Reranqueamento usando um modelo estilo-BERT com o treinamento no dataset do MS MARCO e avaliação no TREC-DL 2020


O treinamento é igual ao de um classificador binário, que será feito por vocês.


O que muda é a forma de avaliação: reranqueadores precisam ser alimentados com documentos candidatos (ex: trazidos pelo BM25 - exercício aula 1)


Sugestões:
1. usar este dataset reduzido do MS MARCO como treinamento, com 10k triplas (query, passagem relevante, passagem não-relevante):
https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv


2. usar miniLM (modelo BERT pequeno, 5x mais rapido) para começar o finetuning: https://huggingface.co/nreimers/MiniLM-L6-H384-uncased pois oferece um bom compromisso entre qualidade e velocidade.


3. usar este notebook como base
Análise de sentimentos (dataset IMDB) usando um modelo estilo BERT: https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing


4. para debug: usar este minilm para ver se consegue ndcg ~0.70: https://huggingface.co/cross-encoder/ms-marco-MiniLM-L-6-v2Sugestão: fazer overfit em um batch: treinar por 200 epocas um unico batch, e ver se consegue loss=0, e acuracia=100%, ou ndcg=1



# Organizando o ambiente

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Wed Mar 15 22:08:53 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P0    28W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory


In [3]:
def mostra_memoria(lista_mem=['cpu']):
  """
  Esta função exibe informações de memória da CPU e/ou GPU, conforme parâmetros fornecidos.

  Parâmetros:
  -----------
  lista_mem : list, opcional
      Lista com strings 'cpu' e/ou 'gpu'. 
      'cpu' - exibe informações de memória da CPU.
      'gpu' - exibe informações de memória da GPU (se disponível).
      O valor padrão é ['cpu'].

  Saída:
  -------
  A função não retorna nada, apenas exibe as informações na tela.

  Exemplo de uso:
  ---------------
  Para exibir informações de memória da CPU:
      mostra_memoria(['cpu'])

  Para exibir informações de memória da CPU e GPU:
      mostra_memoria(['cpu', 'gpu'])
  
  Autor: Marcus Vinícius Borela de Castro

  """  
  if 'cpu' in lista_mem:
    vm = virtual_memory()
    ram={}
    ram['total']=round(vm.total / 1e9,2)
    ram['available']=round(virtual_memory().available / 1e9,2)
    # ram['percent']=round(virtual_memory().percent / 1e9,2)
    ram['used']=round(virtual_memory().used / 1e9,2)
    ram['free']=round(virtual_memory().free / 1e9,2)
    ram['active']=round(virtual_memory().active / 1e9,2)
    ram['inactive']=round(virtual_memory().inactive / 1e9,2)
    ram['buffers']=round(virtual_memory().buffers / 1e9,2)
    ram['cached']=round(virtual_memory().cached/1e9 ,2)
    print(f"Your runtime RAM in gb: \n total {ram['total']}\n available {ram['available']}\n used {ram['used']}\n free {ram['free']}\n cached {ram['cached']}\n buffers {ram['buffers']}")
    print('/nGPU')
    gpu_info = !nvidia-smi
  if 'gpu' in lista_mem:
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
      print('Not connected to a GPU')
    else:
      print(gpu_info)


In [4]:
mostra_memoria(['cpu'])

Your runtime RAM in gb: 
 total 27.33
 available 25.8
 used 1.17
 free 22.61
 cached 3.18
 buffers 0.37
/nGPU


### Vinculando pasta do google drive para salvar dados

In [5]:
import os

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
current_dir = os.getcwd()
print("Current directory:", current_dir)

Current directory: /content


Ainda não usei opção abaixo: 

Controle de estágio de execução do caderno

O desenvolvimento passa por fases: estagio_1, dados, treino, estagio_2, completo

A ideia é bypassar algumas execuções se não estiver na fase atual do projeto.

lista_fase_atual 

lista_estado_atual = ['estagio_1', 'dados', 'treino', 'estagio_2']

lista_estado_atual = ['dados']

## Fixando as seeds

In [8]:
import random
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader

In [9]:
def inicializa_seed(num_semente:int=123):
  """
  Inicializa as sementes para garantir a reprodutibilidade dos resultados do modelo.
  Essa é uma prática recomendada, já que a geração de números aleatórios pode influenciar os resultados do modelo.
  Além disso, a função também configura as sementes da GPU para garantir a reprodutibilidade quando se utiliza aceleração por GPU. 
  
  Args:
      num_semente (int): número da semente a ser utilizada para inicializar as sementes das bibliotecas.
  
  References:
      http://nlp.seas.harvard.edu/2018/04/03/attention.html
      https://github.com/CyberZHG/torch-multi-head-attention/blob/master/torch_multi_head_attention/multi_head_attention.py#L15
  """
  # Define as sementes das bibliotecas random, numpy e pytorch
  random.seed(num_semente)
  np.random.seed(num_semente)
  torch.manual_seed(num_semente)
  
  # Define as sementes da GPU
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = False

  #torch.cuda.manual_seed(num_semente)
  #Cuda algorithms
  #torch.backends.cudnn.deterministic = True


In [10]:
num_semente=123
inicializa_seed(num_semente)

## Definindo Hiperparâmetros iniciais

In [11]:
def inicia_hparam()->dict:
  # Inicialização dos parâmetros
  hparam = {}
  hparam["num_workers_dataloader"] = 0
  hparam["device"] = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
  if torch.cuda.is_available(): print(torch. cuda. get_device_name(hparam["device"]))    
  return hparam

In [12]:
hparam=inicia_hparam()

Tesla T4


## Preparando para debug e display

In [13]:
import pandas as pd

In [14]:
mostra_memoria(('cpu','gpu'))

Your runtime RAM in gb: 
 total 27.33
 available 25.72
 used 1.25
 free 22.04
 cached 3.67
 buffers 0.37
/nGPU
Wed Mar 15 22:09:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    29W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [15]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 105.9 MB/s eta 0:00:00


In [16]:
import transformers

In [17]:
mostra_memoria(('cpu','gpu'))

Your runtime RAM in gb: 
 total 27.33
 available 25.69
 used 1.28
 free 21.84
 cached 3.84
 buffers 0.37
/nGPU
Wed Mar 15 22:09:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    29W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

https://zohaib.me/debugging-in-google-collab-notebook/

In [18]:
!pip install -Uqq ipdb
import ipdb
# %pdb off # desativa debug em exceção
# %pdb on  # ativa debug em exceção
# ipdb.set_trace(context=8)  para execução nesse ponto

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 793.3/793.3 KB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 385.8/385.8 KB 41.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires ipython~=7.9.0, but you have ipython 8.11.0 which is incompatible.


In [215]:
def config_display():
  """
  Esta função configura as opções de display do Pandas.
  """

  # Configurando formato saída Pandas
  # define o número máximo de colunas que serão exibidas
  pd.options.display.max_columns = None

  # define a largura máxima de uma linha
  pd.options.display.width = 1000

  # define o número máximo de linhas que serão exibidas
  pd.options.display.max_rows = 100

  # define o número máximo de caracteres por coluna
  pd.options.display.max_colwidth = 50

  # se deve exibir o número de linhas e colunas de um DataFrame.
  pd.options.display.show_dimensions = True

  # número de dígitos após a vírgula decimal a serem exibidos para floats.
  pd.options.display.precision = 7


In [20]:
def config_debug():
  """
  Esta função configura as opções de debug do PyTorch e dos pacotes
  transformers e datasets.
  """

  # Define opções de impressão de tensores para o modo científico
  torch.set_printoptions(sci_mode=True) 
  """
    Significa que valores muito grandes ou muito pequenos são mostrados em notação científica.
    Por exemplo, em vez de imprimir o número 0.0000012345 como 0.0000012345, 
    ele seria impresso como 1.2345e-06. Isso é útil em situações em que os valores dos tensores 
    envolvidos nas operações são muito grandes ou pequenos, e a notação científica permite 
    uma melhor compreensão dos números envolvidos.  
  """

  # Habilita detecção de anomalias no autograd do PyTorch
  torch.autograd.set_detect_anomaly(True)
  """
    Permite identificar operações que podem causar problemas de estabilidade numérica, 
    como gradientes explodindo ou desaparecendo. Quando essa opção é ativada, 
    o PyTorch verifica se há operações que geram valores NaN ou infinitos nos tensores 
    envolvidos no cálculo do gradiente. Se for detectado um valor anômalo, o PyTorch 
    interrompe a execução e gera uma exceção, permitindo que o erro seja corrigido 
    antes que se torne um problema maior.

    É importante notar que a detecção de anomalias pode ter um impacto significativo 
    no desempenho, especialmente em modelos grandes e complexos. Por esse motivo,
    ela deve ser usada com cautela e apenas para depuração.
  """

  # Configura variável de ambiente para habilitar a execução síncrona (bloqueante) das chamadas da API do CUDA.
  os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
  """
    o Python aguarda o término da execução de uma chamada da API do CUDA antes de executar a próxima chamada. 
    Isso é útil para depurar erros no código que envolve operações na GPU, pois permite que o erro seja capturado 
    no momento em que ocorre, e não depois de uma sequência de operações que pode tornar a origem do erro mais difícil de determinar.
    No entanto, é importante lembrar que esse modo de execução é significativamente mais lento do que a execução assíncrona, 
    que é o comportamento padrão do CUDA. Por isso, é recomendado utilizar esse comando apenas em situações de depuração 
    e removê-lo após a solução do problema.
  """

  # Define o nível de verbosity do pacote transformers para info
  transformers.utils.logging.set_verbosity_info() 
  
  
  """
    Define o nível de detalhamento das mensagens de log geradas pela biblioteca Hugging Face Transformers 
    para o nível info. Isso significa que a biblioteca irá imprimir mensagens de log informativas sobre
    o andamento da execução, tais como tempo de execução, tamanho de batches, etc.

    Essas informações podem ser úteis para entender o que está acontecendo durante a execução da tarefa 
    e auxiliar no processo de debug. É importante notar que, em alguns casos, a quantidade de informações
    geradas pode ser muito grande, o que pode afetar o desempenho do sistema e dificultar a visualização
    das informações relevantes. Por isso, é importante ajustar o nível de detalhamento de acordo com a 
    necessidade de cada tarefa.
  
    Caso queira reduzir a quantidade de mensagens, comentar a linha acima e 
      descomentar as duas linhas abaixo, para definir o nível de verbosity como error ou warning
  
    transformers.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_warning()
  """


  # Define o modo verbose do xmode, que é utilizado no debug
  %xmode Verbose 

  """
    Comando usado no Jupyter Notebook para controlar o modo de exibição das informações de exceções.
    O modo verbose é um modo detalhado que exibe informações adicionais ao imprimir as exceções.
    Ele inclui as informações de pilha de chamadas completa e valores de variáveis locais e globais 
    no momento da exceção. Isso pode ser útil para depurar e encontrar a causa de exceções em seu código.
    Ao usar %xmode Verbose, as informações de exceção serão impressas com mais detalhes e informações adicionais serão incluídas.

    Caso queira desabilitar o modo verbose e utilizar o modo plain, 
    comentar a linha acima e descomentar a linha abaixo:
    %xmode Plain
  """

  """
    Dica:
    1.  pdb (Python Debugger)
      Quando ocorre uma exceção em uma parte do código, o programa para a execução e exibe uma mensagem de erro 
      com informações sobre a exceção, como a linha do código em que ocorreu o erro e o tipo da exceção.

      Se você estiver depurando o código e quiser examinar o estado das variáveis ​​e executar outras operações 
      no momento em que a exceção ocorreu, pode usar o pdb (Python Debugger). Para isso, é preciso colocar o comando %debug 
      logo após ocorrer a exceção. Isso fará com que o programa pare na linha em que ocorreu a exceção e abra o pdb,
      permitindo que você explore o estado das variáveis, examine a pilha de chamadas e execute outras operações para depurar o código.


    2. ipdb
      O ipdb é um depurador interativo para o Python que oferece recursos mais avançados do que o pdb,
      incluindo a capacidade de navegar pelo código fonte enquanto depura.
      
      Você pode começar a depurar seu código inserindo o comando ipdb.set_trace() em qualquer lugar do 
      seu código onde deseja pausar a execução e começar a depurar. Quando a execução chegar nessa linha, 
      o depurador entrará em ação, permitindo que você examine o estado atual do seu programa e execute 
      comandos para investigar o comportamento.

      Durante a depuração, você pode usar comandos:
        next (para executar a próxima linha de código), 
        step (para entrar em uma função chamada na próxima linha de código) 
        continue (para continuar a execução normalmente até o próximo ponto de interrupção).

      Ao contrário do pdb, o ipdb é um depurador interativo que permite navegar pelo código fonte em que
      está trabalhando enquanto depura, permitindo que você inspecione variáveis, defina pontos de interrupção
      adicionais e até mesmo execute expressões Python no contexto do seu programa.
  """


In [216]:
config_display()

In [22]:
config_debug()

Exception reporting mode: Verbose


## Rastro (neptune.ai)

Gerado rastro da execução no Neptune (detalhes no artigo [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664))


### Importação de libraries para Rastro

In [23]:
!pip install neptune-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.8/443.8 KB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 KB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.9/840.9 KB 32.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 99.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 KB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.8/136.8 KB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66

In [24]:
import neptune

/usr/local/lib/python3.9/dist-packages/neptune/internal/backends/hosted_client.py:48: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version


In [25]:
!pip install torchviz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4148 sha256=fd4b3f513b4526cf794e1e85e1d4e1754e1478f52fec72f28d5ddec0b38a4357
  Stored in directory: /root/.cache/pip/wheels/29/65/6e/db2515eb1dc760fecd36b40d54df65c1e18534013f1c037e2e
Successfully built torchviz


In [26]:
from torchviz import make_dot 

In [27]:
import getpass
import copy
import time
import re
import tempfile

### Código Rastro

Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664), autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)



In [28]:
def converte_optimizer_state_dict(parm_optimizer)-> dict:
  """
    Recebe um objeto "parm_optimizer" que é do tipo "torch.optim.Optimizer" e retorna um dicionário 
    com informações sobre o otimizador.

    O dicionário de retorno é gerado a partir do estado do otimizador que é extraído da propriedade
    "state_dict()" do objeto "parm_optimizer", seu primeiro grupo de parâmetros do otimizador.
  """
  # return str(hparam['optimizer'])
  return parm_optimizer.state_dict()['param_groups'][0]

In [29]:
def gera_tag_rastro_experiencia_treino(parm_aula: str, hparam: dict) -> str:
    """
    Gera uma string formatada com informações de hiperparâmetros para ser usada como tag de rastro de experiência de treino.

    Args:
        parm_aula (str): Nome da aula que está sendo treinada.
        hparam (dict): Dicionário contendo os hiperparâmetros utilizados no treinamento.

    Returns:
        str: String formatada com as informações de hiperparâmetros.

    Uso: 

    hparam['lista_tag_rastro_experiencia_treino'] =        gera_tag_rastro_experiencia_treino(parm_aula='aula7', hparam=hparam)
    """
    # Inicializa uma lista vazia para armazenar as tags
    lista_tag = []
    
    # Lista com as chaves dos hiperparâmetros que serão utilizados
    lista_chaves = ['embed_dim', 'leiaute_input', 'dim_feedforward', 'max_seq_length', 'ind_activation_function', 'batch_size', 'learning_rate', 'weight_decay', 'amsgrad', 'decrease_factor_lr', 'max_examples', 'eval_every_steps']
    
    # Itera pelas chaves da lista e cria uma string com a chave e o valor correspondente em hparam,
    # adicionando essa string à lista_tag
    for chave in lista_chaves:
        if chave in hparam:
          tag = f"{chave} {hparam[chave]}"
          lista_tag.append(tag)
    
    # Concatena a lista de tags em uma única string, separando cada tag por '|',
    # e adicionando o nome da aula como prefixo
    tag_formatada = f"{parm_aula}|" + "|".join(lista_tag)
    
    return tag_formatada




In [30]:
class NeptuneRastroRun():
    """
      Classe para geração de rastro de experimento utilizando a ferramenta Neptune.

      Busca implementar o rastro proposto em [Rastro-DM: Mineração de Dados com Rastro](https://revista.tcu.gov.br/ojs/index.php/RTCU/article/view/1664),
      autores Marcus Vinícius Borela de Castro e Remis Balaniuk, com o apoio da [solução Neptune](https://app.neptune.ai/)

      Attributes:
      -----------
      se_geracao_rastro : bool
          Indica se deve ser gerado rastro de experimento. 
      neptune_project : str
          Nome do projeto criado no Neptune. 
      tag_contexto_rastro : str
          Nome da tag utilizada para identificar o experimento.
      neptune_api_token : str
          Token utilizado para autenticação na API do Neptune. 
      run_neptune : object
          Objeto que representa o experimento no Neptune.
      device : str
          Dispositivo utilizado para o treinamento do modelo.
      tmpDir : str
        Diretório temporário utilizado para salvar gráfico do modelo.          
    """
    se_geracao_rastro = True 
    neptune_project = ""
    tag_contexto_rastro = ""
    neptune_api_token = ""

    def __init__(self, parm_params:dict,  parm_lista_tag:list = None):
      """
        Método construtor da classe NeptuneRastroRun.
        
        Args:
        - parm_params: dicionário contendo os parâmetros do modelo.
        - parm_lista_tag: lista contendo tags adicionais para o experimento.
      """      
      # print(f"NeptuneRastroRun.init: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_params `{parm_params} ")
      if self.__class__.se_geracao_rastro:      
        self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)
        self.run_neptune['sys/name'] = self.__class__.tag_contexto_rastro
        vparams = copy.deepcopy(parm_params)
        if "optimizer" in vparams:
          vparams["optimizer"] = converte_optimizer_state_dict(vparams["optimizer"])
        if 'criterion'  in vparams:
          vparams["criterion"] = str(vparams["criterion"])
        if 'scheduler'  in vparams:
          vparams["scheduler"] = str(type(vparams["scheduler"]))
        if 'device' in vparams:
          vparams['device'] = str(vparams["device"])
        self.device = vparams["device"]
        for tag in parm_lista_tag:
          self.run_neptune['sys/tags'].add(tag)
        self.run_neptune['parameters'] = vparams
        # self.tmpDir = tempfile.mkdtemp()

    @property
    def run():
      """
      Retorna a instância do objeto run_neptune.
      """      
      return self.run_neptune

    @classmethod
    def ativa_geracao_rastro(cls):
      """
      Ativa a geração de rastro.
      """      
      cls.se_geracao_rastro = True      

    @classmethod
    def def_contexto(cls):
      """
      Define o contexto para a geração de rastro.
      """      
      cls.se_geracao_rastro = True      

    @classmethod
    def desativa_geracao_rastro(cls):
      """
      Desativa a geração de rastro.
      """      
      cls.se_geracao_rastro = False      

    @classmethod
    def retorna_status_geracao_rastro(cls):
      """
        Retorna o status da geração de rastro.
        
        Returns:
        - True se a geração de rastro está ativada, False caso contrário.
      """      
      return cls.se_geracao_rastro      

    @classmethod
    def retorna_tag_contexto_rastro(cls):
      """
        Retorna a tag do contexto de rastro.
      """      
      return cls.tag_contexto_rastro 

    @classmethod
    def inicia_contexto(cls, neptune_project, tag_contexto_rastro, neptune_api_token):
      """
      Inicia o contexto de execução no Neptune.

      Args:
          neptune_project (str): Nome do projeto no Neptune.
          tag_contexto_rastro (str): Tag que identifica o contexto de execução no Neptune.
          neptune_api_token (str): Token de acesso à API do Neptune.

      Raises:
          AssertionError: Caso a tag_contexto_rastro possua um ponto (.), 
            o que pode gerar erros na gravação de arquivo.
      """      
      assert '.' not in tag_contexto_rastro, "NeptuneRastroRun.init(): tag_contexto_rastro não pode possuir ponto, pois será usado para gravar nome de arquivo"      
      cls.neptune_api_token = neptune_api_token
      cls.tag_contexto_rastro = tag_contexto_rastro
      cls.neptune_project = neptune_project

    def salva_metrica(self, parm_metricas={}):
      """
        Salva as métricas no Neptune Run caso a geração de rastro esteja ativa.

        Parameters
        ----------
        parm_metricas: dict
            Dicionário contendo as métricas a serem salvas. As chaves devem ser os nomes das métricas e os valores devem ser
            os valores das métricas.
      """
      #print(f"NeptuneRastroRun.salva_metrica: se_geracao_rastro {self.__class__.se_geracao_rastro} parm_metricas:{parm_metricas} ")
      if self.__class__.se_geracao_rastro:
        for metrica, valor in parm_metricas.items(): 
          self.run_neptune[metrica].append(valor)
 
    def gera_grafico_modelo(self, loader_train, model):
      """
        Gera um gráfico do modelo e o envia para o Neptune. 
        Para gerar o gráfico, um forward pass é realizado em um batch de exemplos 
        de treino e o resultado é renderizado como um gráfico de nós conectados. 
        O gráfico é salvo em um arquivo .png e enviado para o Neptune como um arquivo anexo.

        Args:
            loader_train (torch.utils.data.DataLoader): DataLoader do conjunto de treinamento.
            model (torch.nn.Module): Modelo a ser visualizado.
        
        Pendente:
          Evolui para usar from io import StringIO (buffer = io.StringIO()) ao invés de tempdir 
      """      
      if self.__class__.se_geracao_rastro: 
        # efetuar um forward 
        """
        se dataloader devolver x e y:
        """
        x_, y_ = next(iter(loader_train))
        x_ = x_.to(self.device)
        outputs = model(x_)
        """
        # se dataloader devolver dict:
        dados_ = next(iter(loader_train))
        outputs = model(dados_['x'].to(self.device))
        #outputs = model(x_['input_ids'].to(self.device), x_['attention_mask'].to(self.device))
        """
        nome_arquivo = os.path.join(self.tmpDir, "modelo "+ self.__class__.tag_contexto_rastro + time.strftime("%Y-%b-%d %H:%M:%S"))
        make_dot(outputs, params=dict(model.named_parameters()), show_attrs=True, show_saved=True).render(nome_arquivo, format="png")
        self.run_neptune["parameters/model_graph"].upload(nome_arquivo+'.png')
        self.run_neptune['parameters/model'] = re.sub('<bound method Module.state_dict of ', '',str(model.state_dict))      



    def stop(self):
      """
        Para a execução do objeto Neptune. Todos os experimentos do Neptune são sincronizados com o servidor, e nenhum outro 
        experimento poderá ser adicionado a este objeto após a chamada a este método.
      """
      if self.__class__.se_geracao_rastro:         
        self.run_neptune.stop()


### Definindo parâmetros para o rastro



In [31]:
NeptuneRastroRun.inicia_contexto('marcusborela/IA386DD', 'Aula 2 - classificador de texto como reranqueador',   getpass.getpass('Informe NEPTUNE_API_TOKEN'))
#NeptuneRastroRun.desativa_geracao_rastro()

Informe NEPTUNE_API_TOKEN··········


## Configurando Pyserini

### Instalações de libraries

In [32]:
!pip install pyserini

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.1/137.1 MB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 KB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 KB 13.6 MB/s eta 0:00:00
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [33]:
!pip install faiss-cpu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 67.4 MB/s eta 0:00:00


### Baixando o repositório do pyserini para usara seus scripts

In [34]:
path_pyserini = '/content/drive/MyDrive/treinamento/202301_IA368DD/code/pyserini'
path_pyserini_tools = path_pyserini + '/pyserini-master/anserini-tools-master'
path_pyserini_eval = path_pyserini + '/pyserini-master/pyserini/eval'

In [35]:
if not os.path.exists(path_pyserini):
    os.makedirs(path_pyserini)
    print('pasta criada')
    !wget -q https://github.com/castorini/pyserini/archive/refs/heads/master.zip -O pyserini.zip 
    !unzip -q pyserini.zip -d  {path_pyserini}
    # Baixando tools que é um atalho para https://github.com/castorini/anserini-tools
    !wget -q https://github.com/castorini/anserini-tools/archive/refs/heads/master.zip -O anserini-tools.zip 
    !unzip -q anserini-tools.zip -d  {path_pyserini}


In [36]:
path_pyserini = path_pyserini + '/pyserini-master'

In [37]:
assert os.path.exists(path_pyserini), f"Pasta {path_pyserini} não criada!"

In [38]:
assert os.path.exists(path_pyserini_tools), f"Pasta {path_pyserini_tools} não criada!"

In [39]:
assert os.path.exists(path_pyserini_eval), f"Pasta {path_pyserini_eval} não criada!"

# Carga dos dados da TREC 2020 usando pyserini

### Obtendo dados dos documentos a partir do pyserini


[Dicas aqui](https://github.com/castorini/pyserini/blob/master/docs/experiments-msmarco-passage.md)

In [40]:
path_data = '/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage'

In [41]:
%%time
if not os.path.exists(path_data):
  os.makedirs(path_data)
  print('pasta criada')
  !wget https://msmarco.blob.core.windows.net/msmarcoranking/collectionandqueries.tar.gz -P {path_data}
  !tar xvfz {path_data}/collectionandqueries.tar.gz -C {path_data}
  os.remove(f'{path_data}/collectionandqueries.tar.gz')
  print("Dados carregados!")
else:
  print("Dados já existiam!")    

Dados já existiam!
CPU times: user 2.06 ms, sys: 0 ns, total: 2.06 ms
Wall time: 1.68 ms


In [42]:
assert os.path.exists(path_data), f"Pasta {path_data} não criada!"

Passo anterior gera os seguintes arquivos:

* collection.tsv
* qrels.dev.small.tsv
* qrels.train.tsv
* queries.dev.small.tsv
* queries.dev.tsv
* queries.eval.small.tsv
* queries.eval.tsv
* queries.train.tsv

In [43]:
%%time
if not os.path.exists(f'{path_data}/collection_json'):
  !python {path_pyserini_tools}/tools/scripts/msmarco/convert_collection_to_jsonl.py \
  --collection-path {path_data}/collection.tsv \
  --output-folder {path_data}/collection_json
  print("Dados carregados!")
else:
  print("Dados já existiam!")    

Dados já existiam!
CPU times: user 1.75 ms, sys: 496 µs, total: 2.25 ms
Wall time: 351 ms


In [44]:
assert os.path.exists(f'{path_data}/collection_json'), f"Pasta {path_data}/collection_json não criada!"

The above script should generate 9 json files in collections/msmarco-passage/collection_json, each with 1M lines (except for the last one, which should have 841,823 lines). ]

Cada linha tem um documento com id e contents (texto)

### Loading data in dicts

The 6980 queries in the development set are already stored in the repo. Let's take a peek:

In [45]:
!head {path_pyserini_tools}/topics-and-qrels/topics.msmarco-passage.dev-subset.txt

1048585	what is paula deen's brother
2	 Androgen receptor define
524332	treating tension headaches without medication
1048642	what is paranoid sc
524447	treatment of varicose veins in legs
786674	what is prime rate in canada
1048876	who plays young dr mallard on ncis
1048917	what is operating system misconfiguration
786786	what is priority pass
524699	tricare service number


In [46]:
!head {path_pyserini_tools}/topics-and-qrels/topics.dl20.txt

1030303	who is aziz hashim
1037496	who is rep scalise?
1043135	who killed nicholas ii of russia
1045109	who owns barnhart crane
1049519	who said no one can make you feel inferior
1051399	who sings monk theme song
1056416	who was the highest career passer  rating in the nfl
1064670	why do hunters pattern their shotguns?
1065636	why do some places on my scalp feel sore
1071750	why is pete rose banned from hall of fame


In [47]:
!head {path_data}/qrels.dev.small.trec

300674 0 7067032 1
125705 0 7067056 1
94798 0 7067181 1
9083 0 7067274 1
174249 0 7067348 1
320792 0 7067677 1
1090270 0 7067796 1
1101279 0 7067891 1
201376 0 7068066 1
54544 0 7068203 1


In [48]:
!head {path_pyserini_tools}/topics-and-qrels/qrels.dl20-passage.txt

23849 0 1020327 2
23849 0 1034183 3
23849 0 1120730 0
23849 0 1139571 1
23849 0 1143724 0
23849 0 1147202 0
23849 0 1150311 0
23849 0 1158886 2
23849 0 1175024 1
23849 0 1201385 0


Each line contains a tab-delimited (query id, query) pair. Conveniently, Pyserini already knows how to load and iterate through these pairs. We can now perform retrieval using these queries:

#### Carregando qrels (relevância por query)

##### Carregando usando get_qrels

In [49]:
from pyserini.search import get_qrels

In [50]:
qrels = get_qrels('dl20-passage')

In [51]:
len(qrels)

54

In [202]:
qrels

{23849: {1020327: '2',
  1034183: '3',
  1120730: '0',
  1139571: '1',
  1143724: '0',
  1147202: '0',
  1150311: '0',
  1158886: '2',
  1175024: '1',
  1201385: '0',
  1215556: '0',
  1220759: '0',
  1221770: '0',
  1333480: '1',
  1381453: '2',
  1414114: '2',
  1414115: '0',
  1414120: '2',
  1449780: '0',
  146754: '0',
  1493231: '0',
  1532701: '0',
  1535484: '0',
  1605854: '1',
  1605857: '1',
  1622747: '1',
  17118: '0',
  17122: '0',
  1714915: '0',
  1714917: '1',
  1724687: '0',
  172488: '0',
  178252: '0',
  182049: '0',
  1827512: '1',
  1844627: '0',
  188190: '0',
  188246: '1',
  1944730: '0',
  2003292: '0',
  2017213: '0',
  2203364: '0',
  2209883: '0',
  2318793: '0',
  2339898: '1',
  2373852: '0',
  2397072: '0',
  2423771: '0',
  2516458: '0',
  2585563: '0',
  2593928: '0',
  2607127: '3',
  2607128: '1',
  2607129: '2',
  2607130: '2',
  2607131: '2',
  2607132: '3',
  2607134: '0',
  2647769: '3',
  2674124: '0',
  2766280: '0',
  282421: '0',
  2838462: '

In [52]:
list(qrels.items())[0]

(23849,
 {1020327: '2',
  1034183: '3',
  1120730: '0',
  1139571: '1',
  1143724: '0',
  1147202: '0',
  1150311: '0',
  1158886: '2',
  1175024: '1',
  1201385: '0',
  1215556: '0',
  1220759: '0',
  1221770: '0',
  1333480: '1',
  1381453: '2',
  1414114: '2',
  1414115: '0',
  1414120: '2',
  1449780: '0',
  146754: '0',
  1493231: '0',
  1532701: '0',
  1535484: '0',
  1605854: '1',
  1605857: '1',
  1622747: '1',
  17118: '0',
  17122: '0',
  1714915: '0',
  1714917: '1',
  1724687: '0',
  172488: '0',
  178252: '0',
  182049: '0',
  1827512: '1',
  1844627: '0',
  188190: '0',
  188246: '1',
  1944730: '0',
  2003292: '0',
  2017213: '0',
  2203364: '0',
  2209883: '0',
  2318793: '0',
  2339898: '1',
  2373852: '0',
  2397072: '0',
  2423771: '0',
  2516458: '0',
  2585563: '0',
  2593928: '0',
  2607127: '3',
  2607128: '1',
  2607129: '2',
  2607130: '2',
  2607131: '2',
  2607132: '3',
  2607134: '0',
  2647769: '3',
  2674124: '0',
  2766280: '0',
  282421: '0',
  2838462: 

#### Carregando queries

##### Carregando usando get_topics

In [53]:
from pyserini.search import get_topics

In [54]:
topics = get_topics('dl20')
print(f'{len(topics)} queries total')

200 queries total


In [55]:
len(topics), list(topics.items())[0]

(200, (735922, {'title': 'what is crimp oil'}))

Deixando apenas queries com relevância

In [56]:
topics = {key:value for key, value in topics.items() if key in qrels}

In [57]:
len(topics), list(topics.items())[0]

(54, (23849, {'title': 'are naturalization records public information'}))

# Executando 1o estágio: exact match com BM25

In [58]:
num_max_hits = 1000

#### Com código para retornar o score do bm25

In [59]:
from tqdm import tqdm

In [60]:
# Run all queries in topics, retrive top 1k for each query
def run_all_queries(file, topics, searcher, num_max_hits=100):
  """
  A função run_all_queries é responsável por realizar todas as consultas armazenadas no dicionário topics utilizando o objeto searcher fornecido e salvar os resultados em um arquivo de texto.

  Parâmetros:

  file: caminho do arquivo de saída onde serão salvos os resultados das consultas.
  topics: dicionário contendo as consultas a serem executadas. Cada consulta é representada por uma chave única no dicionário. O valor correspondente a cada chave é um outro dicionário contendo as informações da consulta, como seu título e outras informações relevantes.
  searcher: objeto do tipo Searcher que será utilizado para realizar as consultas.
  num_max_hits: número máximo de documentos relevantes que serão retornados para cada consulta.
  Retorno:

  A função não retorna nenhum valor, mas salva os resultados das consultas no arquivo especificado em file.
  Comentário:

  A função usa a biblioteca tqdm para exibir uma barra de progresso enquanto executa as consultas.
  O número de consultas concluídas é impresso a cada 100 consultas.
  """
  print(f'Running {len(topics)} queries in total')
  with open(file, 'w') as runfile:
      cnt = 0
      for id in tqdm(topics, desc='Running Queries'):
          query = topics[id]['title']
          hits = searcher.search(query, num_max_hits)
          for i in range(0, len(hits)):
              _ = runfile.write(f'{id} Q0 {hits[i].docid} {i+1} {hits[i].score:.6f} Etapa_EM\n')
              # = runfile.write('{} Q0 {} {} {:.6f} Pyserini\n'.format(id, hits[i].docid, i+1, hits[i].score))
          cnt += 1
          if cnt % 100 == 0:
              print(f'{cnt} queries completed')

In [61]:
path_run = path_data + '/runs'
path_run_estagio1 = path_run + '/run-msmarco-passage-bm25-estagio1.txt'

In [62]:
path_run, path_run_estagio1

('/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/runs',
 '/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/runs/run-msmarco-passage-bm25-estagio1.txt')

In [63]:
%%time
if not os.path.exists(path_run):
  os.makedirs(path_run)
  print('pasta criada!')
else:
  print('pasta já existia!')


pasta já existia!
CPU times: user 64 µs, sys: 825 µs, total: 889 µs
Wall time: 1.98 ms


In [64]:
if not os.path.exists(path_run_estagio1):
  # roda 1o estágio de busca por bm25
  # code from https://colab.research.google.com/github/castorini/anserini-notebooks/blob/master/pyserini_msmarco_passage_demo.ipynb
  from pyserini.search import SimpleSearcher
  from pyserini.search.lucene import LuceneSearcher


  # Indexando Trec 2020 Collection using Pyserini
  !python -m pyserini.index.lucene \
    --collection JsonCollection \
    --input {path_data}/collection_json \
    --index indexes/lucene-index-msmarco-passage \
    --generator DefaultLuceneDocumentGenerator \
    --threads 9 \
    --storePositions --storeDocvectors --storeRaw

  # realizando busca e salvando dados do estágio 1
  searcher = LuceneSearcher('./indexes/lucene-index-msmarco-passage')
  searcher.set_bm25(k1=0.82, b=0.68)  
  run_all_queries(path_run_estagio1, topics, searcher, num_max_hits)
  
  print("Dados estágio 1 (bm25) carregados!")
else:
  print("Dados estágio 1 (bm25) já existiam!")    

Dados estágio 1 (bm25) já existiam!


In [65]:
assert os.path.exists(path_run_estagio1), f"Pasta {path_run_estagio1} não criada!"

In [66]:
!head {path_run_estagio1}

23849 Q0 4348282 1 10.738400 Etapa_EM
23849 Q0 2674124 2 10.426600 Etapa_EM
23849 Q0 542113 3 9.687200 Etapa_EM
23849 Q0 8246990 4 9.641500 Etapa_EM
23849 Q0 8133127 5 9.566500 Etapa_EM
23849 Q0 2647769 6 9.547200 Etapa_EM
23849 Q0 2017213 7 9.504800 Etapa_EM
23849 Q0 7119957 8 9.500700 Etapa_EM
23849 Q0 964148 9 9.476800 Etapa_EM
23849 Q0 436721 10 9.464000 Etapa_EM


##### Eval

In [67]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mrecall.1000 -mmap -l 2 'dl20-passage' {path_run_estagio1}

jtreceval-0.0.5-jar-with-dependencies.jar: 1.79MB [00:00, 6.84MB/s]                
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-mrecall.1000', '-mmap', '-l', '2', '/root/.cache/pyserini/topics-and-qrels/qrels.dl20-passage.txt', '/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/runs/run-msmarco-passage-bm25-estagio1.txt']
Results:
map                   	all	0.2876
recall_1000           	all	0.8031
ndcg_cut_10           	all	0.4876


# Preparando dados de treinamento


## Carga dos dados

In [174]:
path_data_train = path_data + '/train'
path_data_train

'/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/train'

In [175]:
%%time
if not os.path.exists(path_data_train):
  os.makedirs(path_data_train)
  print(f'{path_data_train} pasta criada!')
else:
  print(f'{path_data_train} pasta já existia!')

/content/drive/MyDrive/treinamento/202301_IA368DD/collections/msmarco-passage/train pasta já existia!
CPU times: user 0 ns, sys: 959 µs, total: 959 µs
Wall time: 780 µs


In [176]:
assert os.path.exists(path_data_train), f"Path {path_data_train} não existe!"

In [177]:
if not os.path.exists(path_data_train+'/msmarco_triples.train.tiny.tsv'):
  !wget -P {path_data_train} https://storage.googleapis.com/unicamp-dl/ia368dd_2023s1/msmarco/msmarco_triples.train.tiny.tsv  
  print(f'arquivo baixado!')

In [178]:
assert os.path.exists(path_data_train+'/msmarco_triples.train.tiny.tsv'), f"Path {path_data_train+'/msmarco_triples.train.tiny.tsv'} não existe!"

In [179]:
!head -n 2 {path_data_train+'/msmarco_triples.train.tiny.tsv'}

is a little caffeine ok during pregnancy	We donât know a lot about the effects of caffeine during pregnancy on you and your baby. So itâs best to limit the amount you get each day. If youâre pregnant, limit caffeine to 200 milligrams each day. This is about the amount in 1Â½ 8-ounce cups of coffee or one 12-ounce cup of coffee.	It is generally safe for pregnant women to eat chocolate because studies have shown to prove certain benefits of eating chocolate during pregnancy. However, pregnant women should ensure their caffeine intake is below 200 mg per day.
what fruit is native to australia	Passiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rating. Some sources list the fruit as edible, sweet and tasty, while others list the fruits as being bitter and inedible.assiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rating. Some sou

In [180]:
df = pd.read_csv(path_data_train+'/msmarco_triples.train.tiny.tsv', sep='\t', header=None)

In [181]:
df.columns = ['query', 'positive', 'negative']

In [182]:
df.head()

,query,positive,negative
0,is a little caffeine ok during pregnancy,We donât know a lot about the effects of caf...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,"The kola nut is the fruit of the kola tree, a ..."
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...","However, you still need some niacin each day; ..."


In [183]:
df.dtypes


query       object
positive    object
negative    object
Length: 3, dtype: object

Verificando correção do arquivo!

In [184]:
print(df.isnull().sum())

query       0
positive    0
negative    0
Length: 3, dtype: int64


### Tratando qualidade dos dados

#### Ajustando o texto no dataframe

In [185]:
df['query']=df['query'].astype(str)
df['positive']=df['positive'].astype(str)
df['negative']=df['negative'].astype(str)

In [186]:
x = 'We donât know a lot about the effects'
#x = x.encode('utf-8').decode('unicode_escape')
x = x.replace('â', '\'')
print(x)

We don't know a lot about the effects


In [187]:
df['query'] = df['query'].apply(lambda x: x.replace('â', '\''))
df['positive'] = df['positive'].apply(lambda x: x.replace('â', '\''))
df['negative'] = df['negative'].apply(lambda x: x.replace('â', '\''))


In [188]:
df.shape

(11000, 3)

In [189]:
df.head(3)

,query,positive,negative
0,is a little caffeine ok during pregnancy,We don't know a lot about the effects of caffe...,It is generally safe for pregnant women to eat...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,"The kola nut is the fruit of the kola tree, a ..."
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,The Canadian Physician Health Institute (CPHI)...


In [190]:
df['positive'].iloc[0]

"We don't know a lot about the effects of caffeine during pregnancy on you and your baby. So it's best to limit the amount you get each day. If you're pregnant, limit caffeine to 200 milligrams each day. This is about the amount in 1Â½ 8-ounce cups of coffee or one 12-ounce cup of coffee."

Código abaixo não funcionou. A evoluir (se der tempo)!

import unicodedata

x = '1Â½ 8-ounce cups of coffee'

x = unicodedata.normalize("NFKD", x)  # Normaliza a string

print(x)

s = re.sub(r'(\d+)[\u00C2\u00A0]([\u2150-\u215E\u2189])', r'\1\2', x, flags=re.UNICODE)
print(s)  # "1½ 8-ounce cups of coffee"

#### Verificando média de tamanho (em palavras, não em tokens)

In [85]:
df.applymap(len).mean()

query        34.23
positive    353.50
negative    340.20
Length: 3, dtype: float64

## Divisão em treino e validação (de forma estratificada)

In [191]:
df_data = pd.DataFrame()

In [192]:
df_data['query']=df['query']
df_data['passage']=df['positive']
df_data['classe']=1


In [193]:
df[['query','negative']].assign(classe=0).head(3)

,query,negative,classe
0,is a little caffeine ok during pregnancy,It is generally safe for pregnant women to eat...,0
1,what fruit is native to australia,"The kola nut is the fruit of the kola tree, a ...",0
2,how large is the canadian military,The Canadian Physician Health Institute (CPHI)...,0


In [194]:
df_data = df_data.append(df[['query','negative']].rename(columns={'negative': 'passage'}).assign(classe=0), ignore_index=True)

<ipython-input-194-05336a35200b>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_data = df_data.append(df[['query','negative']].rename(columns={'negative': 'passage'}).assign(classe=0), ignore_index=True)


In [195]:
df_data.shape

(22000, 3)

In [196]:
df_data.head(5)

,query,passage,classe
0,is a little caffeine ok during pregnancy,We don't know a lot about the effects of caffe...,1
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...,1
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...,1
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...,1
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,...",1


In [197]:
df_data.tail(5)

,query,passage,classe
21995,where is maramal in skyrim,One type of character isn't necessarily better...,0
21996,which is a basic unit of a sugar molecule,According to Merriam-Webster and the Online Et...,0
21997,deloitte consulting or phone number,Virtual phone numbers are convenient for both ...,0
21998,"how many hours a day to work to get lunch in ,a",1 Levaquin can be taken at any time of the day...,0
21999,what theatre company did shakespeare invest in,"The Globe Theatre was constructed in 1599, out...",0


In [91]:
assert df_data.shape[0] == 22000, f'Erro no df_data.shape[0]. Deveria ser 22000 e é {df_data.shape[0]}'

In [92]:
df_data.head(3), df_data.tail(3)

(                                      query                                            passage  classe
 0  is a little caffeine ok during pregnancy  We don't know a lot about the effects of caffe...       1
 1         what fruit is native to australia  Passiflora herbertiana. A rare passion fruit n...       1
 2        how large is the canadian military  The Canadian Armed Forces. 1  The first large-...       1
 
 [3 rows x 3 columns],
                                                  query                                            passage  classe
 21997              deloitte consulting or phone number  Virtual phone numbers are convenient for both ...       0
 21998  how many hours a day to work to get lunch in ,a  1 Levaquin can be taken at any time of the day...       0
 21999   what theatre company did shakespeare invest in  The Globe Theatre was constructed in 1599, out...       0
 
 [3 rows x 3 columns])

In [93]:
from sklearn.model_selection import train_test_split

In [94]:
hparam['percent_test_size'] = .15 #@param [.10, .15, .20] {type:'raw'}

In [95]:
df_data[['query','passage']]

,query,passage
0,is a little caffeine ok during pregnancy,We don't know a lot about the effects of caffe...
1,what fruit is native to australia,Passiflora herbertiana. A rare passion fruit n...
2,how large is the canadian military,The Canadian Armed Forces. 1 The first large-...
3,types of fruit trees,Cherry. Cherry trees are found throughout the ...
4,how many calories a day are lost breastfeeding,"Not only is breastfeeding better for the baby,..."
...,...,...
21995,where is maramal in skyrim,One type of character isn't necessarily better...
21996,which is a basic unit of a sugar molecule,According to Merriam-Webster and the Online Et...
21997,deloitte consulting or phone number,Virtual phone numbers are convenient for both ...
21998,"how many hours a day to work to get lunch in ,a",1 Levaquin can be taken at any time of the day...


In [96]:
df_data[['query','passage']].values

array([['is a little caffeine ok during pregnancy',
        "We don't know a lot about the effects of caffeine during pregnancy on you and your baby. So it's best to limit the amount you get each day. If you're pregnant, limit caffeine to 200 milligrams each day. This is about the amount in 1Â½ 8-ounce cups of coffee or one 12-ounce cup of coffee."],
       ['what fruit is native to australia',
        'Passiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rating. Some sources list the fruit as edible, sweet and tasty, while others list the fruits as being bitter and inedible.assiflora herbertiana. A rare passion fruit native to Australia. Fruits are green-skinned, white fleshed, with an unknown edible rating. Some sources list the fruit as edible, sweet and tasty, while others list the fruits as being bitter and inedible.'],
       ['how large is the canadian military',
        "The Canadian Armed Forces. 1  T

In [97]:
# hparam['leiaute_input'] = 'qp' # primeiro query, depois passage
# hparam['leiaute_input'] = 'pq' # primeiro passage, depois query
hparam['leiaute_input'] = 'qppq' # primeiro passage, query union query,passage (só no treino, para não impactar o teste de validação, não ter sido visto no treino)

O dataset de validação será sempre no leiaute qp (para permitir comparar)

In [98]:
X_train, X_valid, Y_train, Y_valid = train_test_split(df_data[['query','passage']].values, 
                                                      df_data['classe'].values,
                                                      test_size=hparam['percent_test_size'], 
                                                      stratify=df_data['classe'].values, random_state=num_semente)
if hparam['leiaute_input'] == 'qp':
  pass
elif hparam['leiaute_input'] == 'pq':
  X_train = np.flip(X_train, axis=1)
elif hparam['leiaute_input'] == 'qppq':
  X_train_flip = np.flip(X_train, axis=1)
  X_train = np.concatenate((X_train, X_train_flip), axis=0)
  Y_train = np.concatenate((Y_train, Y_train), axis=0)
else:
  raise Exception(f"hparam['leiaute_input'] inválido {hparam['leiaute_input']}")

In [99]:
X_train.shape

(37400, 2)

In [100]:
X_train.shape, type(X_valid), type(Y_valid), X_valid.shape, Y_valid.shape

((37400, 2), numpy.ndarray, numpy.ndarray, (3300, 2), (3300,))

In [101]:
print(np.unique(Y_train, return_counts=True), '\n', np.unique(Y_valid, return_counts=True))

(array([0, 1]), array([18700, 18700])) 
 (array([0, 1]), array([1650, 1650]))


In [102]:
X_train[:2]

array([['who is nucky thompson based on',
        'Nolan Lyons (1884) Marc Pickering (1897) Enoch Malachi Nucky Thompson, played by Steve Buscemi, is the protagonist of the series Boardwalk Empire. He is very loosely based on the real New Jersey politician Enoch L. Johnson (1883-1968), who was in control of Atlantic City for two decades, between 1911 and 1931.'],
       ['can i bury paper money',
        "They keep your tax return once filed and do NOT return the return to you. Are you asking about your refund? YOU had the problem which resulted in denial of efiling. Paper returns always take longer to process. I just got my federal refund for a paper return filed 4/15 and the state refund hasn't come yet. It takes as long as it takes, and there is nothing you can do about it, venting and ranting won't change anything. The state is out of money, so I don't expect anything anytime soon."]],
      dtype=object)

In [103]:
Y_train[:2]

array([1, 0])

### Tokenizer: carga e computando limites

In [128]:
const_nome_modelo = 'microsoft/MiniLM-L12-H384-uncased'

In [129]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained(const_nome_modelo)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/special_tokens_map.json
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  

In [106]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 25.67
 used 1.27
 free 19.76
 cached 5.92
 buffers 0.39
/nGPU
Wed Mar 15 08:41:02 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [107]:
tokenizer.pad_token_id,tokenizer.cls_token_id,tokenizer.sep_token_id

(0, 101, 102)

In [108]:
x='x, it is just a test!'
y='y, it is just a continuation!'

In [109]:
tokenizer(x,y)['input_ids']

[101,
 1060,
 1010,
 2009,
 2003,
 2074,
 1037,
 3231,
 999,
 102,
 1061,
 1010,
 2009,
 2003,
 2074,
 1037,
 13633,
 999,
 102]

In [110]:
tokenizer.encode_plus(x,y)['input_ids']

[101,
 1060,
 1010,
 2009,
 2003,
 2074,
 1037,
 3231,
 999,
 102,
 1061,
 1010,
 2009,
 2003,
 2074,
 1037,
 13633,
 999,
 102]

In [111]:
df_tokenizado = df.applymap(lambda x: tokenizer(x)['input_ids'])

In [112]:
df_tokenizado.head()

,query,positive,negative
0,"[101, 2003, 1037, 2210, 24689, 7959, 3170, 792...","[101, 2057, 2123, 1005, 1056, 2113, 1037, 2843...","[101, 2009, 2003, 3227, 3647, 2005, 6875, 2308..."
1,"[101, 2054, 5909, 2003, 3128, 2000, 2660, 102]","[101, 3413, 10128, 10626, 2050, 7253, 11410, 1...","[101, 1996, 12849, 2721, 17490, 2003, 1996, 59..."
2,"[101, 2129, 2312, 2003, 1996, 3010, 2510, 102]","[101, 1996, 3010, 4273, 2749, 1012, 1015, 1996...","[101, 1996, 3010, 7522, 2740, 2820, 1006, 1813..."
3,"[101, 4127, 1997, 5909, 3628, 102]","[101, 9115, 1012, 9115, 3628, 2024, 2179, 2802...","[101, 1996, 12849, 2721, 17490, 2003, 1996, 59..."
4,"[101, 2129, 2116, 10250, 18909, 1037, 2154, 20...","[101, 2025, 2069, 2003, 7388, 7959, 17819, 248...","[101, 2174, 1010, 2017, 2145, 2342, 2070, 9152..."


In [113]:
print('max:', '\n', df_tokenizado.applymap(len).max(), '\n','mean:', '\n', df_tokenizado.applymap(len).mean(), '\n', 'std:', '\n', df_tokenizado.applymap(len).std())

max: 
 query        43
positive    280
negative    249
Length: 3, dtype: int64 
 mean: 
 query        9.10
positive    80.88
negative    77.52
Length: 3, dtype: float64 
 std: 
 query        2.85
positive    32.58
negative    31.12
Length: 3, dtype: float64


Logo: max_seq_length pode ser 322 (43 + 280 - 1 [sep comum aos 2])

### Definindo parâmetros e informações para o rastro (hparam) - sobre dados

In [ ]:
# visto que "max_position_embeddings": 512 em https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/blob/main/config.json
hparam['model_name'] = const_nome_modelo
hparam['vocab_size']= tokenizer.vocab_size
hparam['batch_size']=32
hparam['num_epochs'] = 10  #@param {type:"slider", min:1, max:100, step:1} 
hparam['max_seq_length'] = 322  #@param {type:"slider", min:100, max:512, step:1} 
hparam['num_sentenca_train'] = X_train.shape[0] # 18700 ou 37400
hparam['num_sentenca_valid'] = X_valid.shape[0] # 3300


Definindo o prefixo de nome do arquivo 

In [100]:
path_aula = '/content/drive/MyDrive/treinamento/202301_IA368DD/aula2/'
if not os.path.exists(path_aula):
  os.makedirs(path_aula)
  print(f'{path_aula} pasta criada!')
else: 
  print(f'{path_aula} pasta já existia!')
  

/content/drive/MyDrive/treinamento/202301_IA368DD/aula2/ pasta já existia!


In [101]:
infixo_nome= 'rerank_max_seq_'+ str(hparam['max_seq_length'])+'_text_'
path_treino = path_aula + infixo_nome
print(path_treino)

/content/drive/MyDrive/treinamento/202301_IA368DD/aula2/rerank_max_seq_322_text_


## Criando código para o Dataset

In [126]:
class MyDataset():
    """
      Classe para representar um dataset de texto e classes.
    """  
    def __init__(self, texts: np.ndarray, classes:list[int], tokenizer, max_seq_length: int):
      """
      Inicializa um novo objeto MyDataset.

      Args:
          texts (np.ndarray): um array com as strings de texto. Cada linha deve ter 2 strings.
          classes (np.ndarray): um array com as classes de cada texto.
          tokenizer: um objeto tokenizer do Hugging Face Transformers.
          max_seq_length (int): o tamanho máximo da sequência a ser considerado.
      Raises:
          AssertionError: se os parâmetros não estiverem no formato esperado.
      """
      # Verifica se os parâmetros são do tipo esperado
      assert isinstance(texts, np.ndarray), f"Parâmetro texts deve ser do tipo np.ndarray e não {type(texts)}"
      assert texts.shape[1] == 2, "Array must have 2 columns"
      for row in texts:
          assert isinstance(row, np.ndarray) and row.shape == (2,), f"Each row in texts must have 2 elements"
          assert isinstance(row[0], str) and isinstance(row[1], str), f"Each element in texts.row must be a string e não {type(row[0])}"
      assert isinstance(classes,np.ndarray), f'classes deve ser do tipo np.ndarray e não {type(classes)}'
      assert isinstance(classes[0],np.int64), f'classes[0] deve ser do tipo numpy.int64 e não {type(classes[0])} '
      assert isinstance(max_seq_length, int), f'max_seq_length deve ser do tipo int e não {type(max_seq_length)}'
      assert max_seq_length > 100, f'max_seq_length deve ser maior do que 100'

      self.texts = texts
      self.classes = classes
      self.tokenizer = tokenizer
      self.max_seq_length = max_seq_length

      # Salvar os dados dos tensores
      x_data_input_ids = []
      x_data_token_type_ids = []
      x_data_attention_masks = []
      for text_pair in tqdm(texts, desc='encoding text pair'):
          encoding = tokenizer.encode_plus(
              text_pair[0],
              text_pair[1],
              add_special_tokens=True,
              max_length=self.max_seq_length,
              padding='max_length',
              return_tensors = 'pt',
              truncation=True,
              return_attention_mask=True,
              return_token_type_ids=True
          )
          x_data_input_ids.append(encoding['input_ids'].long())
          x_data_token_type_ids.append(encoding['token_type_ids'].long())
          x_data_attention_masks.append(encoding['attention_mask'].long())
      print(F'\tVou converter lista para tensor;  Momento: {time.strftime("[%Y-%b-%d %H:%M:%S]")}')
      # squeeze: vai transformar um tensor de shape [2, 1, 322] em um tensor de shape [2, 322].

      self.x_tensor_input_ids = torch.stack(x_data_input_ids).squeeze(1)
      self.x_tensor_attention_masks = torch.stack(x_data_attention_masks).squeeze(1)
      self.x_tensor_token_type_ids = torch.stack(x_data_token_type_ids).squeeze(1)
    def __len__(self):
        """
          Retorna o tamanho do dataset (= tamanho do array texts)
        """
        return len(self.texts)
    
    def __getitem__(self, idx):
        """
          Retorna um dicionário com os dados do texto e sua classe correspondente, em um formato que pode 
          ser usado pelo dataloader do PyTorch para alimentar um modelo de aprendizado de máquina.
        """
        return {
            'input_ids': self.x_tensor_input_ids[idx],
            'attention_mask': self.x_tensor_attention_masks[idx],
            'token_type_ids': self.x_tensor_token_type_ids[idx],
            'labels': int(self.classes[idx])
        }

#### Testando o MyDataset e o Dataloader

In [118]:
# Cria dados fictícios
texts = np.array([['This is the first text', 'This is the second text'],
                  ['This is text 2.1', 'This is text 2.2'],
                  ['This is text 3.1', 'This is text 3.2'],
                  ['This is text 4.1', 'This is text 4.2'],
                  ['This is text 5.1', 'This is text 5.2'],
                  ['This is text 6.1', 'This is text 6.2'],
                  ['This is text 7.1', 'This is text 7.2']])
classes = np.array([1, 0, 1, 0, 1, 0, 1])


In [119]:
# Cria um objeto da classe MyDataset
dummy_dataset = MyDataset(texts=texts, classes=classes, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])

encoding text pair: 100%|██████████| 7/7 [00:00<00:00, 199.07it/s]

	Vou converter lista para tensor;  Momento: [2023-Mar-15 08:41:41]


In [120]:
# Testa o método __len__()
assert len(dummy_dataset) == 7

# Testa o método __getitem__()
sample = dummy_dataset[0]
assert set(sample.keys()) == {'input_ids', 'attention_mask', 'token_type_ids', 'labels'} # 
assert isinstance(sample['input_ids'], torch.Tensor)
assert sample['input_ids'].shape[0] == hparam['max_seq_length']
assert isinstance(sample['attention_mask'], torch.Tensor)
assert sample['attention_mask'].shape[0] == hparam['max_seq_length']
assert isinstance(sample['labels'], int)

In [121]:
print(sample)

{'input_ids': tensor([ 101, 2023, 2003, 1996, 2034, 3793,  102, 2023, 2003, 1996, 2117, 3793,
         102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0, 

In [122]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers=hparam['num_workers_dataloader'] )

In [123]:
mostra_memoria(['cpu','gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 25.61
 used 1.33
 free 19.67
 cached 5.94
 buffers 0.39
/nGPU
Wed Mar 15 08:41:46 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [124]:
first_batch = next(iter(dummy_loader))

In [125]:
first_batch

{'input_ids': tensor([[ 101, 2023, 2003, 1996, 2034, 3793,  102, 2023, 2003, 1996, 2117, 3793,
           102,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
   

Confirmando efeito do parâmetro shuffle em DataLoader

In [126]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=False, num_workers=hparam['num_workers_dataloader'])
for ndx, epoch in tqdm(enumerate(range(hparam['num_epochs'])), desc='Epochs'):
  for batch in dummy_loader:
    print(batch['labels'])
  print(f"Fim época {ndx}")


Epochs: 10it [00:00, 420.81it/s]

tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 0
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 1
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 2
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 3
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 4
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 5
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 6
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 7
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 8
tensor([1, 0])
tensor([1, 0])
tensor([1, 0])
tensor([1])
Fim época 9


In [127]:
dummy_loader = DataLoader(dummy_dataset, batch_size=2, shuffle=True, num_workers=hparam['num_workers_dataloader'])
for ndx, epoch in tqdm(enumerate(range(hparam['num_epochs'])), desc='Epochs'):
  for batch in dummy_loader:
    print(batch['labels'])
  print(f"Fim época {ndx}")

Epochs: 10it [00:00, 415.25it/s]

tensor([0, 1])
tensor([1, 1])
tensor([0, 0])
tensor([1])
Fim época 0
tensor([0, 1])
tensor([0, 1])
tensor([1, 1])
tensor([0])
Fim época 1
tensor([0, 0])
tensor([0, 1])
tensor([1, 1])
tensor([1])
Fim época 2
tensor([1, 0])
tensor([1, 1])
tensor([0, 0])
tensor([1])
Fim época 3
tensor([1, 0])
tensor([0, 1])
tensor([1, 0])
tensor([1])
Fim época 4
tensor([1, 1])
tensor([1, 1])
tensor([0, 0])
tensor([0])
Fim época 5
tensor([1, 0])
tensor([1, 1])
tensor([0, 1])
tensor([0])
Fim época 6
tensor([0, 1])
tensor([0, 1])
tensor([1, 0])
tensor([1])
Fim época 7
tensor([0, 0])
tensor([1, 1])
tensor([1, 0])
tensor([1])
Fim época 8
tensor([1, 0])
tensor([1, 1])
tensor([1, 0])
tensor([0])
Fim época 9


In [128]:
del dummy_loader, dummy_dataset

# Treinamento do ranqueador

Fonte: https://colab.research.google.com/drive/10etP7Lb915EC-uEuf1IKC8DYkyg_om6-?usp=sharing

In this notebook, we will finetuned and evaluate a BERT-based model on a document classification task.

Our dataset will be a smaller version of the IMDB Sentiment Analysis dataset.

To finetune and evaluate faster, we will be using [miniLM](https://huggingface.co/microsoft/MiniLM-L12-H384-uncased), which is small BERT-like model distilled from a larger one.

## Carregando o Dataset e os Dataloaders


Independente da configuração do leiaute de treinamento, a validação é no leiaute qp

In [129]:
datasets_carregados_previamente = False #@param {type:"boolean"}

print(datasets_carregados_previamente)

False


In [130]:
import io

In [366]:
if datasets_carregados_previamente:
  with open(path_treino+'dataset_valid_input_layout_qp_len_'+str(hparam['num_sentenca_valid'])+'.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_valid = torch.load(buffer)
  with open(path_treino+'dataset_train_'+'input_layout_'+hparam['leiaute_input']+'_len_'+str(hparam['num_sentenca_train'])+'.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_train = torch.load(buffer)
  print('Datasets carregados')   

Datasets carregados


In [132]:
%%time
if not datasets_carregados_previamente:
  print("carregando valid_dataset")  
  dataset_valid = MyDataset(texts=X_valid, classes=Y_valid, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(dataset_valid, path_treino+'dataset_valid_input_layout_qp_len_'+str(hparam['num_sentenca_valid'])+'.pt')

carregando valid_dataset


encoding text pair: 100%|██████████| 3300/3300 [00:01<00:00, 2005.13it/s]


	Vou converter lista para tensor;  Momento: [2023-Mar-15 08:44:51]
CPU times: user 1.73 s, sys: 20 ms, total: 1.75 s
Wall time: 2.1 s


In [131]:
%%time
if not datasets_carregados_previamente:
  print("carregando train_dataset")  
  dataset_train = MyDataset(texts=X_train, classes=Y_train, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])
  torch.save(dataset_train, path_treino+'dataset_train_'+'input_layout_'+hparam['leiaute_input']+'_len_'+str(hparam['num_sentenca_train'])+'.pt')

carregando train_dataset


encoding text pair: 100%|██████████| 37400/37400 [00:21<00:00, 1761.85it/s]


	Vou converter lista para tensor;  Momento: [2023-Mar-15 08:44:01]
CPU times: user 21.7 s, sys: 548 ms, total: 22.3 s
Wall time: 22.3 s


In [133]:
dataloader_train = DataLoader(dataset_train, batch_size=hparam['batch_size'], shuffle=True)
dataloader_valid = DataLoader(dataset_valid, batch_size=hparam['batch_size'], shuffle=False)

## Finetuning

### Funções auxiliares de treinamento

In [134]:
gera_tag_rastro_experiencia_treino(parm_aula='aula2', hparam=hparam) 


'aula2|leiaute_input qppq|max_seq_length 322|batch_size 32'

In [135]:
def treina_modelo (model, hparam, parm_dataloader_train, parm_dataloader_valid, parm_se_salva_modelo:bool=True, parm_se_gera_rastro:bool=True):
  global path_treino

  if parm_se_gera_rastro:
      rastro_neptune = NeptuneRastroRun(hparam, parm_lista_tag= gera_tag_rastro_experiencia_treino(parm_aula='aula2', hparam=hparam) )
  history = []
  n_examples = 0
  time_inicio_treino = time.time()

  try:
      best_metrica_validacao = 0 # float('-inf')
      best_epoch = 0
      

      for epoch in tqdm(range(hparam['num_epochs']), desc='Epochs'):
        model.train()

        train_losses = []
        correct = 0
        for batch in tqdm(parm_dataloader_train, mininterval=0.5, desc='Train', disable=False):
            hparam['optimizer'].zero_grad()
            # ipdb.set_trace(context=3)
            outputs = model(input_ids=batch['input_ids'].to(hparam['device']), attention_mask=batch['attention_mask'].to(hparam['device']), token_type_ids=batch['token_type_ids'].to(hparam['device']), labels=batch['labels'].to(hparam['device']))
            loss = outputs.loss
            loss.backward()
            hparam['optimizer'].step()
            hparam['scheduler'].step()
            n_examples += len(batch['input_ids'])  # Increment of batch size
            train_losses.append(loss.cpu().item())

            preds = outputs.logits.argmax(dim=1)
            correct += (preds == batch['labels'].to(hparam['device'])).sum().item()    

        if isinstance(parm_dataloader_train,torch.utils.data.dataloader.DataLoader): 
            accuracy_train = correct / len(parm_dataloader_train.dataset)
        elif isinstance(parm_dataloader_train,list):  # usado para overfit em um batch
            accuracy_train = correct / len(parm_dataloader_train[0]['labels'])    
            assert len(parm_dataloader_train) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"

        loss_train_mean = mean(train_losses)

        accuracy_valid, mean_loss_valid = evaluate(model=model, dataloader=parm_dataloader_valid, set_name='Valid')

        
        print(f'Epoch: {epoch + 1} Training loss: {loss_train_mean:0.2f} accuracy: {accuracy_train:0.3f} Em validação: loss: {mean_loss_valid:0.3f}; accuracy: {accuracy_valid:0.3f}')
        metrica_rastro = {"train/loss": loss_train_mean, 
                          "train/accuracy": accuracy_train, 
                          "train/n_examples": n_examples, 
                          "train/learning_rate": hparam["optimizer"].param_groups[0]["lr"],
                          "valid/loss": mean_loss_valid,
                          "valid/accuracy": accuracy_valid}  
        history.append(metrica_rastro)
        if parm_se_gera_rastro:   
            rastro_neptune.salva_metrica(metrica_rastro)


        if accuracy_valid > best_metrica_validacao:
            best_model_dict = model.state_dict()
            best_metrica_validacao = accuracy_valid
            best_epoch = epoch + 1
            # print(f'encontrado best model Epoch: {epoch + 1} ')
        elif hparam['early_stop'] < (epoch + 1 - best_epoch):
            print(f"Parando por critério de early_stop na época {epoch + 1} sendo best_epoch {best_epoch} e ealy_stop {hparam['early_stop']}")
            break # for epoch


      if parm_se_salva_modelo:
          model.load_state_dict(best_model_dict)
          model.to(hparam['device'])
          path_modelo = f'{path_treino}model_input_layout_{hparam["leiaute_input"]}_len_{hparam["max_seq_length"]}_fim_treino_{time.strftime("[%Y-%b-%d %H:%M:%S]")}.pt'
          torch.save(model, path_modelo)    
          print(f"Modelo com melhor resultado em validação (epoch {best_epoch}) salvo após treino em {path_modelo}")

      # calculando tempo gasto e médio por 
      tempo_treino = time.time() - time_inicio_treino   
      if parm_se_gera_rastro:
          rastro_neptune.run_neptune["context/tempo_treino"] = tempo_treino
          rastro_neptune.run_neptune["context/best_epoch"] = best_epoch
          rastro_neptune.run_neptune["context/tempo_treino_epoc_mean"] = tempo_treino/hparam['num_epochs']
          rastro_neptune.gera_grafico_modelo(parm_dataloader_train, model)  
  finally: # para não deixar em aberto a execução no Neptune
      if parm_se_gera_rastro:
          rastro_neptune.stop()
  return history

In [136]:
def count_parameters(model):
    """
    Conta o número de parâmetros treináveis em um modelo PyTorch.

    Args:
        model (torch.nn.Module): O modelo PyTorch a ser avaliado.

    Returns:
        int: O número de parâmetros treináveis do modelo.

    """
    # Retorna a soma do número de elementos em cada tensor de parâmetro que requer gradiente.
    # A propriedade "requires_grad" é definida como True para todos os tensores de parâmetro que 
    # precisam ser treinados, enquanto que para aqueles que não precisam, ela é definida como False.
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [137]:
from statistics import mean, stdev

In [138]:
# We first define the evaluation function to measure accuracy and loss
def evaluate(model, dataloader, set_name):
    losses = []
    correct = 0
    model.eval()
    with torch.no_grad():
        for ndx, batch in tqdm(enumerate(dataloader), mininterval=0.5, desc=set_name, disable=False):
            input_ids = batch['input_ids'].to(hparam['device'])
            token_type_ids = batch['token_type_ids'].to(hparam['device'])
            attention_mask = batch['attention_mask'].to(hparam['device'])                                              
            labels = batch['labels'].to(hparam['device'])                                              
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
            loss_val = outputs.loss
            losses.append(loss_val.cpu().item())
            preds = outputs.logits.argmax(dim=1)
            correct += (preds == labels).sum().item()
            # if ndx < 5:
            #  print(f'Em evaluete. Batch {ndx}')
            #  print(f'loss_val: {loss_val} ')
            #  print(f'preds: {preds} \nlabels: {labels} ')
            #  print(f'correct: {correct}')
    # ipdb.set_trace(context=3)
    if isinstance(dataloader,torch.utils.data.dataloader.DataLoader): 
        accuracy = correct / len(dataloader.dataset)
    elif isinstance(dataloader,list):  # usado para overfit em um batch
        accuracy = correct / len(dataloader[0]['labels'])
        assert len(dataloader) == 1, f"Calculo de accuracy para overfit em um batch precisa ser revisado"
    mean_loss = mean(losses)
    return accuracy, mean_loss

In [139]:
from torch import nn
from torch import optim
from tqdm.auto import tqdm
from transformers import get_linear_schedule_with_warmup

In [140]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 24.95
 used 1.99
 free 18.34
 cached 6.62
 buffers 0.39
/nGPU
Wed Mar 15 08:45:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [141]:
model = AutoModelForSequenceClassification.from_pretrained(hparam['model_name'], num_labels=2).to(hparam['device'])

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.26.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [142]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 23.46
 used 3.46
 free 15.6
 cached 7.89
 buffers 0.39
/nGPU
Wed Mar 15 08:45:23 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W /  70W |    939MiB / 15360MiB |      1%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

Depois tentar definir:hparam[ hparam['dim_feedforward']=256

In [143]:
hparam['learning_rate']=1e-4 # 3e-5 # 1e-3
hparam['num_params'] = count_parameters(model)
print(f"Number of model parameters: {hparam['num_params']}")
hparam['amsgrad']=False
hparam['weight_decay'] = 1e-4


Number of model parameters: 33360770


In [144]:
hparam['early_stop'] = 3

In [145]:
hparam['num_training_steps'] = hparam['num_epochs'] * len(dataloader_train)
hparam['num_warmup_steps'] = int(hparam['num_training_steps']  * 0.1)

In [146]:
hparam['optimizer'] = torch.optim.Adam(model.parameters(), lr=hparam['learning_rate'], weight_decay= hparam['weight_decay'], amsgrad=hparam['amsgrad'])

In [147]:
hparam['scheduler'] = get_linear_schedule_with_warmup(hparam['optimizer'], hparam['num_warmup_steps'], hparam['num_training_steps'])

### Evaluating on the valid dataset before training

In [148]:
accuracy, mean_loss = evaluate(model=model, dataloader=dataloader_valid, set_name='Valid')
print(f'Em validação: loss: {mean_loss:0.3f}; accuracy: {accuracy:0.3f}')

Valid: 0it [00:00, ?it/s]

Em validação: loss: 0.694; accuracy: 0.500


In [150]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 23.01
 used 3.92
 free 14.66
 cached 8.37
 buffers 0.39
/nGPU
Wed Mar 15 08:46:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    28W /  70W |   1581MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

### Testando em poucos dados: um batch apenas. Esperado overfit loss próxima de zero.

In [151]:
hparam['num_epochs'] = 10
hparam['early_stop'] = 10

In [152]:
history =  treina_modelo (model, hparam,  [next(iter(dataloader_train))], [next(iter(dataloader_valid))], parm_se_salva_modelo=False, parm_se_gera_rastro=False)

Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Train:   0%|          | 0/1 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 1 Training loss: 0.69 accuracy: 0.688 Em validação: loss: 0.702; accuracy: 0.312


Train:   0%|          | 0/1 [00:00<?, ?it/s]

...

Valid: 0it [00:00, ?it/s]

Epoch: 9 Training loss: 0.68 accuracy: 0.688 Em validação: loss: 0.704; accuracy: 0.312


Train:   0%|          | 0/1 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 10 Training loss: 0.68 accuracy: 0.688 Em validação: loss: 0.704; accuracy: 0.312


In [ ]:
raise Exception('Necessário recriar o modelo e objetos de treinamento')

In [ ]:
batch_train1 = next(iter(dataloader_train))
batch_train2 = next(iter(dataloader_train))
batch_valid1 = next(iter(dataloader_valid))
batch_valid2 = next(iter(dataloader_valid))

In [ ]:
history =  treina_modelo (model, hparam,  [batch_train1, batch_train2], [batch_valid1, batch_valid2], parm_se_salva_modelo=False, parm_se_gera_rastro=False)

In [153]:
hparam['early_stop'] = 5

Por algum motivo a investigar quando passo dataset como lista não está gerando aprendizado. Mas quando passo normal, o modelo está aprendendo. 

A solução de passar 2 batchs deve resolver! A confirmar...

In [154]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 22.34
 used 4.57
 free 9.61
 cached 12.65
 buffers 0.51
/nGPU
Wed Mar 15 01:17:05 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    27W /  70W |   9055MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

### Treino do conjunto completo para leiaute_input = 'qppq'

In [154]:
hparam['num_epochs'] = 12

In [155]:
history = treina_modelo (model, hparam,  dataloader_train, dataloader_valid)

<ipython-input-30-38418c3de266>:41: NeptuneWarning: To avoid unintended consumption of logging hours during interactive sessions, the following monitoring options are disabled unless set to 'True' when initializing the run: 'capture_stdout', 'capture_stderr', and 'capture_hardware_metrics'.
  self.run_neptune = neptune.init_run(project=self.__class__.neptune_project, api_token=self.__class__.neptune_api_token, capture_hardware_metrics=True)


https://app.neptune.ai/marcusborela/IA386DD/e/IAD-15


<ipython-input-30-38418c3de266>:55: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'tuple'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  self.run_neptune['parameters'] = vparams
<ipython-input-30-38418c3de266>:55: NeptuneUnsupportedType: You're attempting to log a type that is not directly supported by Neptune (<class 'NoneType'>).
        Convert the value to a supported type, such as a string or float, or use stringify_unsupported(obj)
        for dictionaries or collections that contain unsupported values.
        For more, see https://docs.neptune.ai/help/value_of_unsupported_type
  self.run_neptune['parameters'] = vparams
<ipython-input-30-38418c3de266>:55: NeptuneUnsupportedType: You're attempting to log a ty

Epochs:   0%|          | 0/12 [00:00<?, ?it/s]

Train:   0%|          | 0/1169 [00:00<?, ?it/s]

...

Valid: 0it [00:00, ?it/s]

Epoch: 11 Training loss: 0.00 accuracy: 1.000 Em validação: loss: 0.335; accuracy: 0.942


Train:   0%|          | 0/1169 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 12 Training loss: 0.00 accuracy: 1.000 Em validação: loss: 0.335; accuracy: 0.942
Modelo com melhor resultado em validação (epoch 10) salvo após treino em /content/drive/MyDrive/treinamento/202301_IA368DD/aula2/rerank_max_seq_322_text_model_input_layout_qppq_len_322_fim_treino_[2023-Mar-15 13:11:17].pt
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 9 operations to synchronize with Neptune. Do not kill this process.
All 9 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-15/metadata


In [ ]:
hparam['num_epochs'] = 12

Saida 7 épocas:


Valid:
104/? [00:24<00:00, 4.26it/s]

Epoch: 1 Training loss: 0.36 accuracy: 0.827 Em validação: loss: 0.294; accuracy: 0.877
Train: 100%
1169/1169 [20:35<00:00, 1.02s/it]
Valid:
104/? [00:24<00:00, 4.21it/s]

Epoch: 2 Training loss: 0.17 accuracy: 0.938 Em validação: loss: 0.255; accuracy: 0.912
Train: 100%
1169/1169 [20:59<00:00, 1.01s/it]
Valid:
104/? [00:24<00:00, 4.24it/s]

Epoch: 3 Training loss: 0.08 accuracy: 0.973 Em validação: loss: 0.269; accuracy: 0.928
Train: 100%
1169/1169 [21:16<00:00, 1.11s/it]
Valid:
104/? [00:24<00:00, 4.25it/s]

Epoch: 4 Training loss: 0.04 accuracy: 0.987 Em validação: loss: 0.326; accuracy: 0.934
Train: 37%
433/1169 [08:03<13:09, 1.07s/it]
Valid: 0it [00:00, ?it/s]

Epoch: 5 Training loss: 0.02 accuracy: 0.993 Em validação: loss: 0.269; accuracy: 0.933
Train:   0%|          | 0/1169 [00:00<?, ?it/s]
Valid:
104/? [00:24<00:00, 4.23it/s]

Epoch: 6 Training loss: 0.02 accuracy: 0.995 Em validação: loss: 0.264; accuracy: 0.933
Train: 100%
1169/1169 [20:54<00:00, 1.01it/s]
Valid:
104/? [00:24<00:00, 4.28it/s]

Epoch: 7 Training loss: 0.01 accuracy: 0.997 Em validação: loss: 0.334; accuracy: 0.938

Continuando o treinamento do modelo interrompida por erro de código (rastro: IAD-10, IAD-11).

In [166]:
hparam['num_epochs'] = 5

In [167]:
history = treina_modelo (model, hparam,  dataloader_train, dataloader_valid)

https://app.neptune.ai/marcusborela/IA386DD/e/IAD-11


Epochs:   0%|          | 0/5 [00:00<?, ?it/s]

Train:   0%|          | 0/1169 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 1 Training loss: 0.01 accuracy: 0.999 Em validação: loss: 0.321; accuracy: 0.934


...

Valid: 0it [00:00, ?it/s]

Epoch: 4 Training loss: 0.00 accuracy: 1.000 Em validação: loss: 0.347; accuracy: 0.938


Train:   0%|          | 0/1169 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 5 Training loss: 0.00 accuracy: 1.000 Em validação: loss: 0.347; accuracy: 0.938
Parando por critério de early_stop na época 5 sendo best_epoch 2 e ealy_stop 3
Modelo com melhor resultado em validação (epoch 2) salvo após treino em /content/drive/MyDrive/treinamento/202301_IA368DD/aula2/rerank_max_seq_322_text_model_input_layout_qppq_len_322_fim_treino_[2023-Mar-14 18:21:08].pt
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 9 operations to synchronize with Neptune. Do not kill this process.
All 9 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-11/metadata


### Treino do conjunto completo para leiaute_input = 'qp'

In [ ]:
hparam['num_epochs'] = 10

In [ ]:
history = treina_modelo (model, hparam,  dataloader_train, dataloader_valid)

https://app.neptune.ai/marcusborela/IA386DD/e/IAD-4


Epochs:   0%|          | 0/10 [00:00<?, ?it/s]

Train:   0%|          | 0/585 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 1 Training loss: 0.43 Em validação: loss: 0.286; accuracy: 0.891


...

Valid: 0it [00:00, ?it/s]

Epoch: 9 Training loss: 0.01 Em validação: loss: 0.300; accuracy: 0.939


Train:   0%|          | 0/585 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 10 Training loss: 0.00 Em validação: loss: 0.294; accuracy: 0.939
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 10 operations to synchronize with Neptune. Do not kill this process.
All 10 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-4/metadata


Modelo salvo em: /content/drive/MyDrive/treinamento/202301_IA368DD/aula2/rerank_max_seq_322_text_model_input_layout_qp_len_322_fim_treino_[2023-Mar-13 15:16:12].pt

Com comando:
torch.save(model, path_modelo)  

### Treino do conjunto completo para leiaute_input = 'pq'

In [197]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 21.6
 used 5.32
 free 10.49
 cached 11.05
 buffers 0.47
/nGPU
Tue Mar 14 18:42:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    27W /  70W |   9195MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [198]:
hparam['num_epochs'] = 12
hparam['ealy_stop'] = 5
hparam['leiaute_input'] = 'pq'

In [199]:
history = treina_modelo (model, hparam,  dataloader_train, dataloader_valid)

https://app.neptune.ai/marcusborela/IA386DD/e/IAD-12


Epochs:   0%|          | 0/12 [00:00<?, ?it/s]

Train:   0%|          | 0/1169 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 1 Training loss: 0.36 accuracy: 0.826 Em validação: loss: 0.256; accuracy: 0.906


...

Valid: 0it [00:00, ?it/s]

Epoch: 5 Training loss: 0.02 accuracy: 0.992 Em validação: loss: 0.357; accuracy: 0.933


Train:   0%|          | 0/1169 [00:00<?, ?it/s]

Valid: 0it [00:00, ?it/s]

Epoch: 6 Training loss: 0.02 accuracy: 0.995 Em validação: loss: 0.345; accuracy: 0.911
Parando por critério de early_stop na época 6 sendo best_epoch 3 e ealy_stop 3
Modelo com melhor resultado em validação (epoch 3) salvo após treino em /content/drive/MyDrive/treinamento/202301_IA368DD/aula2/rerank_max_seq_322_text_model_input_layout_pq_len_322_fim_treino_[2023-Mar-14 20:51:27].pt
Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 9 operations to synchronize with Neptune. Do not kill this process.
All 9 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/marcusborela/IA386DD/e/IAD-12/metadata


In [ ]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 22.93
 used 3.99
 free 14.35
 cached 8.56
 buffers 0.44
/nGPU
Tue Mar 14 03:35:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    43W /  70W |   9153MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

# Reranking a partir da saída do BM25

## Agregando informações para a saída do estágio 1

In [86]:
assert os.path.exists(path_run_estagio1), f"Pasta {path_run_estagio1} não criada!"

In [87]:
!head {path_run_estagio1}

23849 Q0 4348282 1 10.738400 Etapa_EM
23849 Q0 2674124 2 10.426600 Etapa_EM
23849 Q0 542113 3 9.687200 Etapa_EM
23849 Q0 8246990 4 9.641500 Etapa_EM
23849 Q0 8133127 5 9.566500 Etapa_EM
23849 Q0 2647769 6 9.547200 Etapa_EM
23849 Q0 2017213 7 9.504800 Etapa_EM
23849 Q0 7119957 8 9.500700 Etapa_EM
23849 Q0 964148 9 9.476800 Etapa_EM
23849 Q0 436721 10 9.464000 Etapa_EM


Necessário reordenar path_run_estagio1 criando path_run_estagio2


Carregar path_run_estagio1 em um dataframe

Gerar dataset a partir dos textos dos pares query + documento, nessa ordem!

Gerar score de todos os registros

Adicionar score_reranking ao dataframe

Regerar arquivo no leiaute de avaliação para etapa_reranking
  Formato (separado por espaço): id_query 'Q0' id_documento ranking score 'Etapa_EM'

avaliar e gerar métricas.

In [88]:
# Carrega o arquivo em um dataframe
df_estagio1 = pd.read_csv(path_run_estagio1, sep=' ', names=['id_query', 'col2', 'id_documento', 'ranking', 'score', 'Etapa_EM'])
# Remove a coluna 'col2', que possui o valor fixo 'Q0'
df_estagio1.drop('col2', axis=1, inplace=True)
df_estagio1.drop('Etapa_EM', axis=1, inplace=True)

In [89]:
df_estagio1.head(4)

,id_query,id_documento,ranking,score
0,23849,4348282,1,10.74
1,23849,2674124,2,10.43
2,23849,542113,3,9.69
3,23849,8246990,4,9.64


In [90]:
df_estagio1['texto_query'] = df_estagio1['id_query'].map(topics).map(lambda x: x['title'])

In [120]:
assert topics[23849]['title'] == 'are naturalization records public information', 'Carga equivocada de queries!'

In [91]:
df_estagio1.iloc[998:1002]

,id_query,id_documento,ranking,score,texto_query
998,23849,8167545,999,7.52,are naturalization records public information
999,23849,8200287,1000,7.52,are naturalization records public information
1000,156498,5606985,1,14.86,do google docs auto save
1001,156498,4273419,2,14.33,do google docs auto save


In [92]:
# verificar a quantidade de memória usada por cada coluna
print(df_estagio1.memory_usage(deep=True))

# verificar a quantidade de memória total ocupada pelo DataFrame
df_estagio1.info(memory_usage='deep')

Index               128
id_query         432000
id_documento     432000
ranking          432000
score            432000
texto_query     4903000
Length: 6, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54000 entries, 0 to 53999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id_query      54000 non-null  int64  
 1   id_documento  54000 non-null  int64  
 2   ranking       54000 non-null  int64  
 3   score         54000 non-null  float64
 4   texto_query   54000 non-null  object 
dtypes: float64(1), int64(3), object(1)
memory usage: 6.3 MB


In [93]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 25.19
 used 1.74
 free 19.23
 cached 5.96
 buffers 0.4
/nGPU
Wed Mar 15 22:58:06 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

In [104]:
import glob

In [105]:
arquivos = glob.glob(f'{path_data}/collection_json/*.json')
dfs = []
for arquivo in tqdm(arquivos, total=len(arquivos)):
    df = pd.read_json(arquivo, lines=True)
    dfs.append(df)

df_documentos = pd.concat(dfs, ignore_index=True)

100%|██████████| 9/9 [01:00<00:00,  6.68s/it]


In [110]:
!head {path_data}/collection_json/docs00.json

{"id": "0", "contents": "The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated."}
{"id": "1", "contents": "The Manhattan Project and its atomic bomb helped bring an end to World War II. Its legacy of peaceful uses of atomic energy continues to have an impact on history and science."}
{"id": "2", "contents": "Essay on The Manhattan Project - The Manhattan Project The Manhattan Project was to see if making an atomic bomb possible. The success of this project would forever change the world forever making it known that something this powerful can be manmade."}
{"id": "3", "contents": "The Manhattan Project was the name for a project conducted during World War II, to develop the first atomic bomb. It refers specifically to t

In [111]:
df_documentos.loc[df_documentos['id'] == 	, 'contents'].values[0]

'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.'

In [112]:
assert df_documentos.loc[df_documentos['id'] == 0	, 'contents'].values[0] == 'The presence of communication amid scientific minds was equally important to the success of the Manhattan Project as scientific intellect was. The only cloud hanging over the impressive achievement of the atomic researchers and engineers is what their success truly meant; hundreds of thousands of innocent lives obliterated.'

In [ ]:
assert df_documentos.loc[df_documentos['id'] == 8167545	, 'contents'].values[0] == 'Verify someone is who they say with criminal history, public records and property info. Get In Touch Get complete contact information, including mobile numbers and addresses.', f"Documento não corresponde ao código"

In [ ]:
df_documentos.shape

(8841823, 2)

In [113]:
del df_documentos

In [106]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 20.33
 used 6.6
 free 4.15
 cached 16.18
 buffers 0.4
/nGPU
Wed Mar 15 23:06:07 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+---

In [231]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 15.89
 used 11.02
 free 0.83
 cached 14.99
 buffers 0.49
/nGPU
Tue Mar 14 22:05:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    27W /  70W |   9195MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+

In [114]:
import gc


In [115]:
# Solicita a coleta de lixo
gc.collect()

322

In [237]:
df_estagio1['texto_documento'] = df_estagio1['id_documento'].map(df_documentos.set_index('id')['contents'])

In [ ]:
df.loc[df['id'] == 1212, 'contents'].values[0]

In [239]:
df_estagio1.iloc[998:1002]

,id_query,id_documento,ranking,score,texto_query,texto_documento
998,23849,8167545,999,7.52,are naturalization records public information,Verify someone is who they say with criminal h...
999,23849,8200287,1000,7.52,are naturalization records public information,Reports regarding the longest recorded sniper ...
1000,156498,5606985,1,14.86,do google docs auto save,"To save it in Google Docs, you had to download..."
1001,156498,4273419,2,14.33,do google docs auto save,"To view the changes from each auto-save, click..."


In [240]:
# verificar a quantidade de memória usada por cada coluna
print(df_estagio1.memory_usage(deep=True))

# verificar a quantidade de memória total ocupada pelo DataFrame
df_estagio1.info(memory_usage='deep')

Index                   128
id_query             432000
id_documento         432000
ranking              432000
score                432000
texto_query         4903000
texto_documento    22475550
Length: 7, dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54000 entries, 0 to 53999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_query         54000 non-null  int64  
 1   id_documento     54000 non-null  int64  
 2   ranking          54000 non-null  int64  
 3   score            54000 non-null  float64
 4   texto_query      54000 non-null  object 
 5   texto_documento  54000 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 27.8 MB


In [241]:
del df_documentos

In [242]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 15.91
 used 11.0
 free 0.84
 cached 15.0
 buffers 0.49
/nGPU
Tue Mar 14 22:10:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    27W /  70W |   9195MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

In [199]:
print(df_estagio1.shape, '\n', df_estagio1.head(5))

(54000, 6) 
    id_query  id_documento  ranking    score                                    texto_query                                    texto_documento
0     23849       4348282        1  10.7384  are naturalization records public information  Civil Records Definition. Civil records are a ...
1     23849       2674124        2  10.4266  are naturalization records public information  See our FAQ's to learn how you can benefit by ...
2     23849        542113        3   9.6872  are naturalization records public information  Public Records â Exemptions and Redactions. ...
3     23849       8246990        4   9.6415  are naturalization records public information  NOTICE OF CONFIDENTIALITY RIGHTS: IF YOU ARE A...
4     23849       8133127        5   9.5665  are naturalization records public information  Spokeo pulls data and information from public ...

[5 rows x 6 columns]


In [ ]:
23849

## Carregando o modelo, o dataset e o dataloader


In [121]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 20.0
 used 6.93
 free 4.02
 cached 15.98
 buffers 0.41
/nGPU
Wed Mar 15 23:17:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8    10W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+--

In [261]:
hparam['leiaute_input'] = 'qp'

In [262]:
hparam['max_seq_length'] = 322
if hparam["leiaute_input"] == 'qp':
    model = torch.load(f'{path_treino}model_input_layout_qp_len_322_fim_treino_[2023-Mar-13 15:16:12].pt')
elif hparam["leiaute_input"] == 'pq':
    model = torch.load(f'{path_treino}model_input_layout_pq_len_322_fim_treino_[2023-Mar-14 20:51:27].pt')    
elif hparam["leiaute_input"] == 'qppq':
    model = torch.load(f'{path_treino}model_input_layout_qppq_len_322_fim_treino_[2023-Mar-15 13:11:17].pt')        

In [263]:
# inicializa o modelo como AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(const_nome_modelo, state_dict=model.state_dict()).to(hparam['device'])


loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--microsoft--MiniLM-L12-H384-uncased/snapshots/44acabbec0ef496f6dbc93adadea57f376b7c0ec/pytorch_model.bin
All model checkpoint wei

In [124]:
datasets_carregados_previamente = False #@param {type:"boolean"}

print(datasets_carregados_previamente)

False


In [ ]:
if datasets_carregados_previamente:
  with open(path_treino+'dataset_reranking_input_layout_qp_len_'+str(hparam['num_sentenca_valid'])+'.pt','rb') as f:
    buffer = io.BytesIO(f.read())
  dataset_reranking = torch.load(buffer)
  print('Datasets carregados')   

In [130]:
%%time
if not datasets_carregados_previamente:
  print("carregando dataset")  
  classes_dummy = np.zeros(df_estagio1.shape[0], dtype=np.int64)
  print(classes_dummy.shape)
  dataset_reranking = MyDataset(texts=df_estagio1[['texto_query','texto_documento']].values, classes=classes_dummy, tokenizer=tokenizer, max_seq_length=hparam['max_seq_length'])    

carregando dataset
(54000,)


encoding text pair: 100%|██████████| 54000/54000 [00:31<00:00, 1740.54it/s]


	Vou converter lista para tensor;  Momento: [2023-Mar-15 23:20:39]
CPU times: user 31.5 s, sys: 424 ms, total: 32 s
Wall time: 31.7 s


In [151]:
df_estagio1['texto_documento'].iloc[0]

'Civil Records Definition. Civil records are a group of public records that pertain to civil registry records, civil family matters and non criminal civil offenses. These records vary a lot because of the nature of the information that is recorded.'

In [134]:
tokenizer.encode_plus(df_estagio1[['texto_query','texto_documento']].values[0])

{'input_ids': [101, 2024, 3019, 3989, 2636, 2270, 2592, 102, 2942, 2636, 6210, 1012, 2942, 2636, 2024, 1037, 2177, 1997, 2270, 2636, 2008, 2566, 18249, 2000, 2942, 15584, 2636, 1010, 2942, 2155, 5609, 1998, 2512, 4735, 2942, 25173, 1012, 2122, 2636, 8137, 1037, 2843, 2138, 1997, 1996, 3267, 1997, 1996, 2592, 2008, 2003, 2680, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [147]:
tokenizer.encode_plus(df_estagio1[['texto_query','texto_documento']].values[1])

{'input_ids': [101, 2024, 3019, 3989, 2636, 2270, 2592, 102, 2156, 2256, 6904, 4160, 1005, 1055, 2000, 4553, 2129, 2017, 2064, 5770, 2011, 2478, 2256, 3945, 1012, 3745, 1024, 5860, 19771, 5017, 1024, 2349, 2000, 1996, 3267, 1997, 1996, 4761, 1997, 2270, 2501, 2592, 1010, 1996, 2270, 2636, 1998, 11088, 2800, 2951, 4216, 2109, 1999, 1996, 2111, 14244, 2291, 2089, 5383, 10697, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [152]:
hparam['batch_size'] = 32

In [257]:
dataloader_reranking = DataLoader(dataset_reranking,
                                  batch_size= hparam['batch_size'], 
                                  shuffle=False)

## Calculando score para pares do estágio 1

In [137]:
mostra_memoria(['cpu', 'gpu'])

Your runtime RAM in gb: 
 total 27.33
 available 17.97
 used 8.95
 free 0.93
 cached 17.05
 buffers 0.41
/nGPU
Wed Mar 15 23:22:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   64C    P0    31W /  70W |    711MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+-

In [251]:
def calcula_relevancia(parm_model, parm_dataloader_reranking):
  prob_relevancia = []
  parm_model.eval()
  # classes_dummy = torch.zeros(hparam['batch_size'], dtype=torch.int64).to(hparam['device'])
  with torch.no_grad():
      for ndx, batch in tqdm(enumerate(parm_dataloader_reranking), total=len(parm_dataloader_reranking), mininterval=0.5, desc='dataset_reranking', disable=False):
          #print("\nbatch['input_ids'][0]", batch['input_ids'][0])
          #print("batch['input_ids'][1]", batch['input_ids'][1])
          outputs = parm_model(input_ids=batch['input_ids'].to(hparam['device']), 
                          attention_mask=batch['token_type_ids'].to(hparam['device']), 
                          token_type_ids=batch['attention_mask'].to(hparam['device']),
                          labels= batch['labels'].to(hparam['device']))
          probs = torch.nn.functional.softmax(outputs.logits, dim=1)
          nao_relevante, relevante = zip(*probs)
          relevantes_float = [float(t.cpu()) for t in relevante]
          prob_relevancia.extend(relevantes_float)
          #print('outpus', outputs)
          #print('probs',probs)
          #print('prob_relevancia',prob_relevancia)
          #break
  return prob_relevancia

Experimentando as 3 configurações de input_leiaute

In [264]:
prob_relevancia_qp = calcula_relevancia(model,dataloader_reranking)

dataset_reranking: 100%|██████████| 1688/1688 [07:08<00:00,  3.94it/s]


In [265]:
prob_relevancia_qp[:10]

[0.9995986819267273,
 0.9880451560020447,
 0.9185846447944641,
 0.02468058094382286,
 0.9982460737228394,
 0.7330121397972107,
 0.9937136769294739,
 0.9973535537719727,
 0.9861835241317749,
 0.8179717063903809]

In [245]:
prob_relevancia_pq = calcula_relevancia(model,dataloader_reranking)

dataset_reranking: 100%|██████████| 1688/1688 [07:07<00:00,  3.95it/s]


In [247]:
prob_relevancia_pq[:10]

[0.9995678067207336,
 0.8363720178604126,
 0.7759460210800171,
 0.0041801040060818195,
 0.9938918352127075,
 0.1556849628686905,
 0.0998314693570137,
 0.9977994561195374,
 0.9365090131759644,
 0.8097450137138367]

Após inicializar o modelo como AutoModelForSequenceClassification

In [252]:
prob_relevancia_qppq = calcula_relevancia(model,dataloader_reranking)

dataset_reranking: 100%|██████████| 1688/1688 [07:08<00:00,  3.94it/s]


In [253]:
prob_relevancia_pq[:10]

[0.9995678067207336,
 0.8363720178604126,
 0.7759460210800171,
 0.0041801040060818195,
 0.9938918352127075,
 0.1556849628686905,
 0.0998314693570137,
 0.9977994561195374,
 0.9365090131759644,
 0.8097450137138367]

In [171]:
prob_relevancia_qppq[:10]

[0.9999445676803589,
 0.9391152858734131,
 0.24496112763881683,
 0.0002424074918963015,
 0.9998745918273926,
 0.9415525197982788,
 0.6408318877220154,
 0.9980884194374084,
 0.004886418581008911,
 0.4354955554008484]

In [266]:
df_estagio1['score_reranking'] = prob_relevancia_qp

Trazendo val_relevancia

In [205]:
val_relevancia = []
for i, row in df_estagio1.iterrows():
    id_query = row['id_query']
    id_documento = row['id_documento']
    if id_query in qrels and id_documento in qrels[id_query]:
        val_relevancia.append(qrels[id_query][id_documento])
    else:
        val_relevancia.append('-1')

In [206]:
df_estagio1['val_relevancia'] = val_relevancia

In [267]:
df_estagio1.head(10)

,id_query,id_documento,ranking,score,texto_query,texto_documento,score_reranking,val_relevancia
0,23849,4348282,1,10.7384,are naturalization records public information,Civil Records Definition. Civil records are a ...,0.9995987,0
1,23849,2674124,2,10.4266,are naturalization records public information,See our FAQ's to learn how you can benefit by ...,0.9880452,0
2,23849,542113,3,9.6872,are naturalization records public information,Public Records â Exemptions and Redactions. ...,0.9185846,0
3,23849,8246990,4,9.6415,are naturalization records public information,NOTICE OF CONFIDENTIALITY RIGHTS: IF YOU ARE A...,0.0246806,1
4,23849,8133127,5,9.5665,are naturalization records public information,Spokeo pulls data and information from public ...,0.9982461,0
5,23849,2647769,6,9.5472,are naturalization records public information,Naturalization Record Search The Naturalizatio...,0.7330121,3
6,23849,2017213,7,9.5048,are naturalization records public information,Notice of Confidentiality Rights: If you are a...,0.9937137,0
7,23849,7119957,8,9.5007,are naturalization records public information,"Yes, in most cases Public Records are availabl...",0.9973536,0
8,23849,964148,9,9.4768,are naturalization records public information,Case Name: Case Number:-2-. would become part ...,0.9861835,-1
9,23849,436721,10,9.4640,are naturalization records public information,Public Records â Exemptions and Redactions. ...,0.8179717,0


In [278]:
# df_estagio1 = pd.read_pickle(f'{path_treino}df_saida_estagio1.pkl')
df_estagio1.to_pickle(f'{path_treino}df_saida_estagio1.pkl')

In [268]:
# Agrupa os registros por id_query e ordena cada grupo pelo score_reranking
df_reranking = df_estagio1.groupby('id_query', group_keys=False).apply(lambda x: x.sort_values(['score_reranking', 'score'], ascending=[False, False]))

In [269]:
df_reranking['reranking'] = df_reranking.groupby('id_query').cumcount() + 1

In [270]:
df_reranking.shape

(54000, 9)

In [271]:
df_reranking.dtypes

id_query             int64
id_documento         int64
ranking              int64
score              float64
texto_query         object
texto_documento     object
score_reranking    float64
val_relevancia      object
reranking            int64
Length: 9, dtype: object

In [272]:
df_reranking.head(15)

,id_query,id_documento,ranking,score,texto_query,texto_documento,score_reranking,val_relevancia,reranking
616,23849,332583,617,7.903500,are naturalization records public information,How to use the Death Records database. Death r...,0.9997744,-1,1
501,23849,7538455,502,8.042164,are naturalization records public information,"About City of Carbon Hill, Walker County, AL P...",0.9997712,-1,2
500,23849,7530907,501,8.042165,are naturalization records public information,"About City of White Plains, Greene County, GA ...",0.9997676,-1,3
319,23849,1963325,320,8.167085,are naturalization records public information,"About City of Harvey, Cook County, IL Public R...",0.9997475,-1,4
314,23849,1607437,315,8.167090,are naturalization records public information,"About Community of Gibsonton, Hillsborough Cou...",0.9997436,-1,5
491,23849,4402160,492,8.042174,are naturalization records public information,"About City of Lake Mary, Seminole County, FL P...",0.9997433,-1,6
802,23849,7124999,803,7.692600,are naturalization records public information,"Health information technicians gather, analyze...",0.9997414,-1,7
976,23849,1805349,977,7.547799,are naturalization records public information,Windham County Recorder of Deeds in Willimanti...,0.9997404,-1,8
818,23849,1280937,819,7.686900,are naturalization records public information,Online background search offer you the opportu...,0.9997346,-1,9
348,23849,6775851,349,8.167056,are naturalization records public information,"About City of Temple, Carroll County, GA Publi...",0.9997345,-1,10


In [273]:
df_reranking[['ranking','score','score_reranking','val_relevancia','reranking']].iloc[980:1020]

,ranking,score,score_reranking,val_relevancia,reranking
936,937,7.588797,0.0004041,-1,981
113,114,8.558999,0.0004030,-1,982
823,824,7.680300,0.0003983,-1,983
649,650,7.842799,0.0003932,-1,984
435,436,8.108998,0.0003890,-1,985
179,180,8.439800,0.0003715,-1,986
847,848,7.665300,0.0003253,-1,987
514,515,8.016300,0.0003211,-1,988
214,215,8.352399,0.0003210,-1,989
575,576,7.954200,0.0002913,-1,990


In [164]:
del df_estagio1

In [277]:
df_reranking.to_pickle(f'{path_treino}df_reranking.pkl')
# df_reranking = pd.read_pickle(f'{path_treino}df_reranking.pkl')

In [222]:
df_reranking.dtypes

id_query             int64
id_documento         int64
ranking              int64
score              float64
texto_query         object
texto_documento     object
score_reranking    float64
val_relevancia      object
reranking            int64
Length: 9, dtype: object

In [223]:
import csv

In [274]:
with open(f'{path_treino}reranking_to_eval.txt', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=' ')
    for row in df_reranking[['id_query', 'id_documento', 'reranking', 'score_reranking']].itertuples(index=False):
        writer.writerow((int(row[0]), 'Q0', int(row[1]), int(row[2]), round(row[3], 6), 'Etapa_EM'))


In [275]:
!head {path_treino}reranking_to_eval.txt

23849 Q0 332583 1 0.999774 Etapa_EM
23849 Q0 7538455 2 0.999771 Etapa_EM
23849 Q0 7530907 3 0.999768 Etapa_EM
23849 Q0 1963325 4 0.999747 Etapa_EM
23849 Q0 1607437 5 0.999744 Etapa_EM
23849 Q0 4402160 6 0.999743 Etapa_EM
23849 Q0 7124999 7 0.999741 Etapa_EM
23849 Q0 1805349 8 0.99974 Etapa_EM
23849 Q0 1280937 9 0.999735 Etapa_EM
23849 Q0 6775851 10 0.999734 Etapa_EM


In [ ]:
23849 Q0 4348282 1 10.738400 Etapa_EM
23849 Q0 2674124 2 10.426600 Etapa_EM
23849 Q0 542113 3 9.687200 Etapa_EM

##### Eval

In [276]:
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 -mrecall.1000 -mmap -l 2 'dl20-passage' {path_treino}reranking_to_eval.txt

/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', '-mrecall.1000', '-mmap', '-l', '2', '/root/.cache/pyserini/topics-and-qrels/qrels.dl20-passage.txt', '/content/drive/MyDrive/treinamento/202301_IA368DD/aula2/rerank_max_seq_322_text_reranking_to_eval.txt']
Results:
map                   	all	0.0287
recall_1000           	all	0.8031
ndcg_cut_10           	all	0.0441


# Observações

Retomei (e aproveitei) conteúdo de cadernos de cursos anteriors na Unicamp

Criei rastro no neptune.ai dos treinamentos.

Usei chat_gpt em todas as fases. Inclusive se mostrou muito útil pra documentar funções. Veja no início: inicializa_seed, config_debug e config_display

Criei a minha classe de Dataset. Nela, gerei os token_type_ids (0 e 1 para as 2 sequências)
Dúvida: Precisa gerar ou o modelo gera para nós quando não passamos?

Experimentei 3 input_layout: qp/pq/qppq (query+passage; passage+query, query+passage union passage+query, com o dobro de registros).
As acurácias na base de validação (fixa, leiaute qp, para comparação e mais próxima do teste no reranking) foram respectivamente: 93,9%, 93,3% e 94,2%

Ajustei erro no apóstrofe no texto de entrada (como em don't)

Dúvida: Não salvei o tokenizador pois ele não foi alterado. Precisa?




# Ressalvas

Na fase do reranking, o ndcg_cut_10 deu baixo 0.0441. Suspeita de causa: não ter usado torch.save(model) ao invés de model.save_pretrained(). Segundo ChatGPT, há uma diferença: model.save_pretrained() salva os pesos do modelo, bem como todos os outros artefatos necessários para carregar e inicializar o modelo em uma instância específica de um modelo do Transformers, enquanto torch.save(model) salva apenas os pesos do modelo.

Obs.: Fiz o ajuste no caderno e deixarei treinando para ver a diferença.
